In [1]:
import numpy as np
import scipy.sparse
import pickle
import xgboost as xgb
import csv
from collections import defaultdict
import redis
import json
import time
import sklearn.utils
import psycopg2
from psycopg2.sql import Identifier, SQL
from datetime import datetime
import os

In [2]:
conn = redis.Redis(password='3oSYTdtZjsuSigRWLcG6VJt9gm4IMvYjQiqsSuGcAc-U4gMNpWGERAevXi9_SHNrn19piz7bBJG0iTLgx7DvknLHTECcHYrqmWb2rsuCWs89svKmhKDD_aMYaXq8IhSeg_89ooPZb0AqLRyR1-fa1zVjrh2UuV0sWFGSk5SjtW0', 
                   host='pangu', port=6379, decode_responses=True)
conn_psql_kongzi2 = psycopg2.connect(database='aida', user='zding', password='dingzishuo', host='localhost',
                                     port=5432)
conn_psql = psycopg2.connect(database='zding', user='zding', password='dingzishuo', host='localhost',
                                     port=5432)

In [3]:
def get_feature_results(exp_id, d_t, data_source='aida_conll'):
    fea_res = conn.lrange('result:::::' + str(exp_id) + ':::::' + data_source + ':::::' + str(d_t), 0, -1)
    fea_vecs = [json.loads(res) for res in fea_res]
    return fea_vecs

def get_dataset_info(exp_id, d_t, data_source='aida_conll', can_size=50):
    valid_mens_size = conn.scard('valid_qry_ids:::::' + str(exp_id) + ':::::' + data_source + ':::::' + str(d_t))
    no_g_can_info_size = conn.scard('no_g_candidate_info:::::' + str(exp_id) + ':::::' + data_source + ':::::' + str(d_t))
    missed_g_candidate_size = conn.scard('missed_g_candidate:::::' + str(exp_id) + ':::::' + data_source + 
                                         ':::::' + str(d_t) + ':::::' + str(can_size))
    no_candidate_size = conn.scard('no_candidate:::::' + str(exp_id) + ':::::' + data_source + ':::::' + str(d_t))
    return valid_mens_size, no_candidate_size, no_g_can_info_size, missed_g_candidate_size

def fetch_all_features(exp_id, data_type, data_source='aida_conll'):
    res_feas = get_feature_results(exp_id, data_type, data_source)
    res_feas_ids = [[res[0].strip('(').split(', ')[0]] + res[1:-1] + [res[-1]] for res in res_feas]
    res_feas_ids = np.array(res_feas_ids, dtype=np.float64)
    return res_feas_ids

def fetch_all_features_delete_max_prior(exp_id, data_type, data_source='aida_conll'):
    res_feas = get_feature_results(exp_id, data_type, data_source)
    res_feas_ids = [[res[0].strip('(').split(', ')[0]] + res[1:2] + [res[2] if res[2]!=0 else res[1]] + res[3:] for res in res_feas]
    res_feas_ids = np.array(res_feas_ids, dtype=np.float64)
    return res_feas_ids

def trans_data(data):
    d_np = data[:, 1:-1]
    #print(d_np)
    d_labels = data[:, -1]
    #print(d_labels)
    idxs = np.where(d_labels == 1)[0]
    d_groups = np.append(np.delete(idxs, 0), len(d_labels)) - idxs
    xgb_data = xgb.DMatrix(data=d_np, label=d_labels)
    xgb_data.set_group(d_groups)
    return xgb_data

def combine_features(original_feas, new_features):
    men_id_feas_dict = defaultdict(list)
    print("Building idx for new features...")
    for fea in new_features:
        men_id_feas_dict[fea[0]].append(fea)
    #for k, v in men_id_feas_dict.items():
    #    print(k)
    #    print(v)
    comb_feas = []
    pre_men_id = 0
    print("Combine original and new features...")
    for fea_idx, fea in enumerate(original_feas):
        #print('Processing: [%d] %s' % (fea_idx, fea))
        if pre_men_id == fea[0]:
            #print('skip')
            continue
        else:
            pre_men_id = fea[0]
            fea_size = len(men_id_feas_dict[fea[0]])
            res = np.append(original_feas[fea_idx: fea_idx + fea_size, :-1], np.array(men_id_feas_dict[fea[0]])[:, 1:], axis=1)
            #print('res:', res)
            comb_feas.append(res)
    return np.concatenate(comb_feas, axis=0)

def evalerror(preds, dt, d_tal_size):
    d_l = dt.get_label()
    idxs = np.where(d_l == 1)[0]
    d_groups = np.append(np.delete(idxs, 0), len(d_l)) - idxs
    matched_ids = []
    q_id = 0
    for x in d_groups:
        pre_res = preds[q_id: x + q_id]
        if(preds[q_id] == max(pre_res)):
            if len([x for x in pre_res if x == preds[q_id]]) == 1:
                matched_ids.append(q_id)
        q_id += x
    precision = float(len(matched_ids)) / len(d_groups)
    recall = float(len(matched_ids)) / d_tal_size
    f1 = 2 * precision * recall / (precision + recall)
    return len(matched_ids), precision, recall, f1

from collections import defaultdict
def evalerror_detail_log(preds, dt, d_tal_size):
    d_l = dt.get_label()
    idxs = np.where(d_l == 1)[0]
    d_groups = np.append(np.delete(idxs, 0), len(d_l)) - idxs
    correct_results = {}
    wrong_results = {}
    duplicates_results = {}
    group_info = {}
    matched_ids = []
    q_id = 0
    for x in d_groups:
        pre_res = preds[q_id: x + q_id]
        if preds[q_id] == max(pre_res):
            correct_results[q_id] = pre_res
            if len([x for i, x in enumerate(pre_res) if x == preds[q_id] ]) == 1:
                matched_ids.append(q_id)
            else:
                duplicates_results[q_id] = pre_res
        else:
            wrong_results[q_id] = pre_res
        q_id += x
    precision = float(len(matched_ids)) / len(d_groups)
    recall = float(len(matched_ids)) / d_tal_size
    f1 = 2 * precision * recall / (precision + recall)
    return len(matched_ids), precision, recall, f1, correct_results, wrong_results, duplicates_results

from collections import defaultdict
from ast import literal_eval
def get_groups_results(preds, dt, res_features, top_k:int=None):
    d_l = dt.get_label()
    idxs = np.where(d_l == 1)[0]
    d_groups = np.append(np.delete(idxs, 0), len(d_l)) - idxs
    correct_res_groups = []
    wrong_res_groups = []
    dup_res_groups = []
    top_k_indices = [] if top_k is not None else None
    q_id = 0
    for x in d_groups:
        pre_res = preds[q_id: x + q_id]
        pre_res_feas = res_features[q_id: x + q_id]
        pred_q_id, pred_ent = literal_eval(res_features[q_id+np.argmax(pre_res)][0])
        
        if preds[q_id] == max(pre_res):
            correct_res_groups.append([pred_q_id, pred_ent])
        else:
            wrong_res_groups.append([pred_q_id, pred_ent])
            
        if top_k is not None:  # save indices of top-k scores in each group
            for i, score in sorted(enumerate(pre_res),key=lambda x:x[1], reverse=True)[:top_k]:
                # print(i, score)
                top_k_indices.append(i+q_id)
        q_id += x
        
    return correct_res_groups, wrong_res_groups, top_k_indices

def fetch_inlinks_by_ent(ent):
    cur = conn_psql_kongzi2.cursor()
    sql = "SELECT _id FROM wikipedia_links_2014 WHERE target=%s;"
    cur.execute(sql, (ent,))
    rows = cur.fetchall()
    cur.close()
    return rows

# fetch an entity's outlinks with duplicates
def fetch_outlinks_by_ent(ent):
    cur = conn_psql_kongzi2.cursor()
    sql = "SELECT target FROM wikipedia_links_2014 WHERE _id=%s;"
    cur.execute(sql, (ent,))
    rows = cur.fetchall()
    cur.close()
    return rows

def fetch_entity_by_mention_emnlp17(mention):
    # print(mention)
    cur = conn_psql_kongzi2.cursor()
    # do a PostgreSQL join to select the entity namestring from the tables dictionary and entity_ids
    sql = "SELECT entity, prior FROM men_ent_dict_emnlp2017 WHERE men_ent_dict_emnlp2017.mention = (E\'%s\') ORDER BY prior DESC;"
    cur.execute(sql % mention.replace("'", "\\'"))
    rows = cur.fetchall()
    cur.close()
    return rows

def fetch_inlinks_redis(ent, link_type='inlinks'):
    inlinks = conn.hmget(link_type, ent)[0]
    return json.loads(inlinks) if inlinks else []

def has_inlinks_redis(ent, link_type='inlinks'):
    inlinks = conn.hexists(link_type, ent)
    return inlinks

def save_inlinks_redis(ent, inlinks, link_type='inlinks'):
    conn.hset(link_type, ent, json.dumps(inlinks))
    
def fetch_outlinks_redis(ent, link_type='outlinks'):
    outlinks = conn.hmget(link_type, ent)[0]
    return json.loads(outlinks) if outlinks else []

def has_outlinks_redis(ent, link_type='outlinks'):
    inlinks = conn.hexists(link_type, ent)
    return inlinks

def save_outlinks_redis(ent, outlinks, link_type='outlinks'):
    conn.hset(link_type, ent, json.dumps(outlinks))
    
def check_links_between_ents(ent_1, ent_2, bidirection=False):
    wiki_pre_str = 'en.wikipedia.org/wiki/'
    inlinks_ent_1 = fetch_inlinks_redis(ent_1, link_type='inlinks')
    if not inlinks_ent_1 and not has_inlinks_redis(ent_1):
        print("PostgreSQL: fetching inlinks for entity {}...".format(ent_1))
        wiki_ents_1 = wiki_pre_str + ent_1
        inlinks_ent_1_db = fetch_inlinks_by_ent(wiki_ents_1)
        inlinks_ent_1 = [x[0].replace(wiki_pre_str, '') for x in inlinks_ent_1_db]
        print("Redis: caching inlinks for entity {}...".format(ent_1))
        save_inlinks_redis(ent_1, inlinks_ent_1)
    inlinks_ent_2 = fetch_inlinks_redis(ent_2, link_type='inlinks')
    if not inlinks_ent_2 and not has_inlinks_redis(ent_2):        
        wiki_ents_2 = wiki_pre_str + ent_2
        print("PostgreSQL: fetching inlinks for entity {}...".format(ent_2))
        inlinks_ent_2_db = fetch_inlinks_by_ent(wiki_ents_2)
        inlinks_ent_2 = [x[0].replace(wiki_pre_str, '') for x in inlinks_ent_2_db]
        print("Redis: caching inlinks for entity {}...".format(ent_2))
        save_inlinks_redis(ent_2, inlinks_ent_2)
    return (ent_1 in inlinks_ent_2 or ent_2 in inlinks_ent_1) if not bidirection else (ent_1 in inlinks_ent_2 and ent_2 in inlinks_ent_1)

def get_links_by_ent(ent, link_type='inlinks'):
    wiki_pre_str = 'en.wikipedia.org/wiki/'
    if link_type == 'inlinks':
        inlinks_ent = fetch_inlinks_redis(ent, link_type='inlinks')
        if not inlinks_ent and not has_inlinks_redis(ent):
#             print("PostgreSQL: fetching inlinks for entity {}...".format(ent))
            wiki_ents = wiki_pre_str + ent
            inlinks_ent_db = fetch_inlinks_by_ent(wiki_ents)
            inlinks_ent = [x[0].replace(wiki_pre_str, '') for x in inlinks_ent_db]
            print("Redis: caching inlinks for entity {}...".format(ent))
            save_inlinks_redis(ent, inlinks_ent)
        return inlinks_ent
    if link_type == 'outlinks':
        outlinks_ent = fetch_outlinks_redis(ent)
        if not outlinks_ent and not has_outlinks_redis(ent):
#             print("PostgreSQL: fetching outlinks for entity {}...".format(ent))
            wiki_ents = wiki_pre_str + ent
            outlinks_ent_db = fetch_outlinks_by_ent(wiki_ents)
            outlinks_ent = [x[0].replace(wiki_pre_str, '') for x in outlinks_ent_db]
            print("Redis: caching outlinks for entity {}...".format(ent))
            save_outlinks_redis(ent, outlinks_ent)
        return outlinks_ent
    
def fetch_ents_by_doc_redis(doc_id):
    id_ents = conn.hmget('doc-predicted-ents-coref-new', doc_id)[0]
    return json.loads(id_ents) if id_ents else []

## Normalized Google Distance
import math
def ngd_similarity(ents_s, ents_t, index_size = 6274625):
    ent_sets_s = set(ents_s)
    ent_sets_t = set(ents_t)
    min_links, max_links = min(len(ent_sets_s), len(ent_sets_t)), max(len(ent_sets_s), len(ent_sets_t))
    com_links = len(ent_sets_s & ent_sets_t)
    if min_links and max_links and com_links:
        return 1 - (math.log(max_links) - math.log(com_links))/ (math.log(index_size) - math.log(min_links))
    else:
        return 0
    
# PMI
def pmi_similarity(ents_s, ents_t, index_size = 6274625, normalize=False):
    ent_sets_s = set(ents_s)
    ent_sets_t = set(ents_t)
    s_links, t_links = len(ent_sets_s), len(ent_sets_t)
    com_links = len(ent_sets_s & ent_sets_t)
    p_s = s_links / index_size
    p_t = t_links / index_size
    p_c = com_links / index_size
    print(p_s, p_t, p_c)
    if p_s and p_t and p_c:
        return p_c/(p_s * p_t) if not normalize else p_c / (p_s * p_t) / min(1/p_s, 1/p_t)
    else:
        return 0
    
        
def get_top_k_prediction_indices(model, d_test, raw_test, docs_dict, top_k):
    d_test_xgboost = trans_data(d_test)
    preds_test = model.predict(d_test_xgboost)
    
    correct_test, wrong_test, top_k_indices_test = get_groups_results(preds_test, d_test_xgboost, raw_test, top_k)
    return top_k_indices_test

In [4]:
d_train_ctx_raw = get_feature_results('basic_fea_ctx', 'train', 'aida_conll')
d_test_a_ctx_raw = get_feature_results('basic_fea_ctx', 'testa', 'aida_conll')
d_test_b_ctx_raw = get_feature_results('basic_fea_ctx', 'testb', 'aida_conll')

In [5]:
print(len(d_train_ctx_raw))
print(len(d_test_a_ctx_raw))
print(len(d_test_b_ctx_raw))

531835
136259
131893


In [6]:
d_train_ctx = fetch_all_features('basic_fea_ctx', 'train', 'aida_conll')
d_test_a_ctx = fetch_all_features('basic_fea_ctx', 'testa', 'aida_conll')
d_test_b_ctx = fetch_all_features('basic_fea_ctx', 'testb', 'aida_conll')
d_train_coref = fetch_all_features('basic_fea_coref', 'train', 'aida_conll')
d_test_a_coref = fetch_all_features('basic_fea_coref', 'testa', 'aida_conll')
d_test_b_coref = fetch_all_features('basic_fea_coref', 'testb', 'aida_conll')

In [7]:
print(d_train_ctx.shape)
print(d_test_a_ctx.shape)
print(d_test_b_ctx.shape)
print(d_train_coref.shape)
print(d_test_a_coref.shape)
print(d_test_b_coref.shape)

(531835, 21)
(136259, 21)
(131893, 21)
(531835, 13)
(136259, 13)
(131893, 13)


In [8]:
model_dir_path = './new_models_14_Aug'
def save_model(model, name):
    if not os.path.exists(model_dir_path):
        os.makedirs(model_dir_path)
    model_path = os.path.join(model_dir_path, '%s.mdl' % name)
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)
        
def load_model(name):
    model_path = os.path.join(model_dir_path, '%s.mdl' % name)
    with open(model_path, 'rb') as f:
        return pickle.load(f)

In [9]:
def get_total_mentions(data_source, data_type) -> int:
    with conn_psql.cursor() as cur:
        sql = SQL("select count(*) from {} where annotation != 'NIL' and type=%s").format(Identifier(data_source))
        cur.execute(sql, (data_type,))
        return cur.fetchone()[0]

In [10]:
d_train_total = get_total_mentions('aida_conll', 'train')
d_test_a_total = get_total_mentions('aida_conll', 'testa')
d_test_b_total = get_total_mentions('aida_conll', 'testb')

print(d_train_total, d_test_a_total, d_test_b_total)

18541 4791 4485


In [11]:
def process(process_name ,train_set, testa_set, testb_set, train_total, testa_total, testb_total, 
            n_estimators, max_depths, train_mode=True, test_a_filter=None, test_b_filter=None,
            eval_func=evalerror_detail_log, train_filter=None):
    dtrain_xgboost = trans_data(train_set)
    dtest_a_xgboost = trans_data(testa_set)
    dtest_b_xgboost = trans_data(testb_set)
    
    if train_filter is not None:
        dtrain_filtered_xgboost = trans_data(train_set[train_filter])

#     train_true_labels = train_set[:, -1].nonzero()[0].size
#     testa_true_labels = testa_set[:, -1].nonzero()[0].size
#     testb_true_labels = testb_set[:, -1].nonzero()[0].size
    
#     print('True labels:', train_true_labels, testa_true_labels, testb_true_labels)
    
    for x in n_estimators:
        num_round = x
        for dep in max_depths:
            model_name = '%d_%d_%s' % (num_round, dep, process_name)
            if train_mode:
                print(datetime.now(), 'Start training')
                param = {'max_depth':dep, 'eta':0.01, 'silent':1, 'objective':'rank:pairwise', 'min_child_weight':0.01}
                if train_filter is not None:
                    bst = xgb.train(param, dtrain_filtered_xgboost, num_round)
                else:
                    bst = xgb.train(param, dtrain_xgboost, num_round)
                print(datetime.now(), 'Training finished')
            else:
                print(datetime.now(), 'Loading model: %s' % model_name)
                bst = load_model(model_name)

            print(datetime.now(), 'Start evaluation')
            preds = bst.predict(dtrain_xgboost)
            a = eval_func(preds, dtrain_xgboost, train_total)
            print("n_estimators: {}, max_depth: {}, acc_training: {}, corr_num: {}".format(num_round, dep, a[2], a[0]))
            
            preds = bst.predict(dtest_a_xgboost)
            if test_a_filter is not None:
                mask = np.ones(len(preds))
                mask[test_a_filter] = 0
                preds -= mask * 1000
            a = eval_func(preds, dtest_a_xgboost, testa_total)
            print("n_estimators: {}, max_depth: {}, acc_validation: {}, corr_num: {}".format(num_round, dep, a[2], a[0]))
            
            preds = bst.predict(dtest_b_xgboost)
            if test_b_filter is not None:
                mask = np.ones(len(preds))
                mask[test_b_filter] = 0
                preds -= mask * 1000
            a = eval_func(preds, dtest_b_xgboost, testb_total)
            print("n_estimators: {}, max_depth: {}, acc_test: {}, corr_num: {}".format(num_round, dep, a[2], a[0]))
            
            print(datetime.now(), 'Evaluation finished')
            
            if train_mode:
                save_model(bst, model_name)
                print(datetime.now(), 'Model Saved: %s' % model_name)

In [12]:
def process_top_ks(bst, filter_preds, testa_set, testb_set, testa_raw, testb_raw, testa_total, testb_total, 
                   top_k_begin, top_k_end, eval_func=evalerror_detail_log):
    dtrain_xgboost = trans_data(train_set)
    dtest_a_xgboost = trans_data(testa_set)
    dtest_b_xgboost = trans_data(testb_set)

    print(datetime.now(), 'Start evaluation')
    
    test_a_preds = bst.predict(dtest_a_xgboost)
    test_b_preds = bst.predict(dtest_b_xgboost)
    
    for top_k in range(top_k_begin, top_k_end):
        print('=== Top K=%d ===' % top_k)
        test_a_filter = get_groups_results(filter_preds, dtest_a_xgboost, testa_raw, top_k)[2]
        test_b_filter = get_groups_results(filter_preds, dtest_b_xgboost, testb_raw, top_k)[2]

        print('test_a_filter', len(test_a_filter))
        print('test_b_filter', len(test_b_filter))

    test_a_preds
    if test_a_filter is not None:
        mask = np.ones(len(preds))
        mask[test_a_filter] = 0
        preds -= mask * 1000
    a = eval_func(preds, dtest_a_xgboost, testa_total)
    print("n_estimators: {}, max_depth: {}, acc_validation: {}, corr_num: {}".format(num_round, dep, a[2], a[0]))

    preds = bst.predict(dtest_b_xgboost)
    if test_b_filter is not None:
        mask = np.ones(len(preds))
        mask[test_b_filter] = 0
        preds -= mask * 1000
    a = eval_func(preds, dtest_b_xgboost, testb_total)
    print("n_estimators: {}, max_depth: {}, acc_test: {}, corr_num: {}".format(num_round, dep, a[2], a[0]))

    print(datetime.now(), 'Evaluation finished')

    if train_mode:
        save_model(bst, model_name)
        print(datetime.now(), 'Model Saved: %s' % model_name)

In [12]:
process('ctx', d_train_ctx, d_test_a_ctx, d_test_b_ctx, d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

/home/ymiao/.venvs/nerd/lib/python3.5/site-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


2019-08-16 15:05:32.192567 Start training
2019-08-16 15:19:44.009347 Training finished
2019-08-16 15:19:44.009833 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9523758157596678, corr_num: 17658
n_estimators: 4900, max_depth: 6, acc_validation: 0.8710081402629931, corr_num: 4173
n_estimators: 4900, max_depth: 6, acc_test: 0.8283166109253066, corr_num: 3715
2019-08-16 15:19:51.671495 Evaluation finished
2019-08-16 15:19:51.728347 Model Saved: 4900_6_ctx


In [9]:
d_train_ctx_no_poly = fetch_all_features('basic_no_poly', 'train', 'aida_conll')
d_test_a_ctx_no_poly = fetch_all_features('basic_no_poly', 'testa', 'aida_conll')
d_test_b_ctx_no_poly = fetch_all_features('basic_no_poly', 'testb', 'aida_conll')

In [10]:
process('ctx_no_poly', d_train_ctx_no_poly, d_test_a_ctx_no_poly, d_test_b_ctx_no_poly, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

/home/ymiao/.venvs/nerd/lib/python3.5/site-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


2019-08-15 23:23:16.806961 Start training
2019-08-15 23:40:02.580252 Training finished
2019-08-15 23:40:02.580735 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9354403753842835, corr_num: 17344
n_estimators: 4900, max_depth: 6, acc_validation: 0.8493007722813609, corr_num: 4069
n_estimators: 4900, max_depth: 6, acc_test: 0.802675585284281, corr_num: 3600
2019-08-15 23:40:10.018825 Evaluation finished
2019-08-15 23:40:10.068225 Model Saved: 4900_6_ctx_no_poly


In [74]:
d_train_ctx_poly = fetch_all_features('basic_poly', 'train', 'aida_conll')
d_test_a_ctx_poly = fetch_all_features('basic_poly', 'testa', 'aida_conll')
d_test_b_ctx_poly = fetch_all_features('basic_poly', 'testb', 'aida_conll')

In [75]:
print(d_train_ctx_poly.shape)
print(d_test_a_ctx_poly.shape)
print(d_test_b_ctx_poly.shape)

(531835, 17)
(136259, 17)
(131893, 17)


In [12]:
process('ctx_poly', d_train_ctx_poly, d_test_a_ctx_poly, d_test_b_ctx_poly, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-15 23:40:27.229694 Start training
2019-08-15 23:58:09.737008 Training finished
2019-08-15 23:58:09.737494 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9325818456393937, corr_num: 17291
n_estimators: 4900, max_depth: 6, acc_validation: 0.841786683364642, corr_num: 4033
n_estimators: 4900, max_depth: 6, acc_test: 0.798216276477146, corr_num: 3580
2019-08-15 23:58:17.585188 Evaluation finished
2019-08-15 23:58:17.627040 Model Saved: 4900_6_ctx_poly


In [13]:
d_train_ctx_poly_no_normalized_tf = fetch_all_features('basic_poly_no_normalized_tf', 'train', 'aida_conll')
d_test_a_ctx_poly_no_normalized_tf = fetch_all_features('basic_poly_no_normalized_tf', 'testa', 'aida_conll')
d_test_b_ctx_poly_no_normalized_tf = fetch_all_features('basic_poly_no_normalized_tf', 'testb', 'aida_conll')

In [14]:
process('ctx_poly_no_normalized_tf', 
        d_train_ctx_poly_no_normalized_tf, 
        d_test_a_ctx_poly_no_normalized_tf, 
        d_test_b_ctx_poly_no_normalized_tf, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-15 23:58:36.968305 Start training
2019-08-16 00:16:31.953042 Training finished
2019-08-16 00:16:31.953540 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9325818456393937, corr_num: 17291
n_estimators: 4900, max_depth: 6, acc_validation: 1.414318513880192, corr_num: 6776
n_estimators: 4900, max_depth: 6, acc_test: 0.798216276477146, corr_num: 3580
2019-08-16 00:16:42.896831 Evaluation finished
2019-08-16 00:16:42.937452 Model Saved: 4900_6_ctx_poly_no_normalized_tf


In [13]:
d_train_ctx_coref  = combine_features(d_train_ctx, d_train_coref)
d_test_a_ctx_coref = combine_features(d_test_a_ctx, d_test_a_coref)
d_test_b_ctx_coref = combine_features(d_test_b_ctx, d_test_b_coref)

Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...


In [14]:
print(d_train_ctx_coref.shape)
print(d_test_a_ctx_coref.shape)
print(d_test_b_ctx_coref.shape)

(531835, 32)
(136259, 32)
(131893, 32)


In [90]:
d_train_ctx_simplified = fetch_all_features('basic_fea_ctx_simplified', 'train', 'aida_conll')
d_test_a_ctx_simplified = fetch_all_features('basic_fea_ctx_simplified', 'testa', 'aida_conll')
d_test_b_ctx_simplified = fetch_all_features('basic_fea_ctx_simplified', 'testb', 'aida_conll')
d_train_coref_simplified = fetch_all_features('basic_fea_coref_simplified', 'train', 'aida_conll')
d_test_a_coref_simplified = fetch_all_features('basic_fea_coref_simplified', 'testa', 'aida_conll')
d_test_b_coref_simplified = fetch_all_features('basic_fea_coref_simplified', 'testb', 'aida_conll')

print(d_train_ctx_simplified.shape)
print(d_test_a_ctx_simplified.shape)
print(d_test_b_ctx_simplified.shape)
print(d_train_coref_simplified.shape)
print(d_test_a_coref_simplified.shape)
print(d_test_b_coref_simplified.shape)

(531835, 17)
(136259, 17)
(131893, 17)
(531835, 10)
(136259, 10)
(131893, 10)


In [91]:
d_train_ctx_coref_simplified  = combine_features(d_train_ctx_simplified, d_train_coref_simplified)
d_test_a_ctx_coref_simplified = combine_features(d_test_a_ctx_simplified, d_test_a_coref_simplified)
d_test_b_ctx_coref_simplified = combine_features(d_test_b_ctx_simplified, d_test_b_coref_simplified)
print(d_train_ctx_coref_simplified.shape)
print(d_test_a_ctx_coref_simplified.shape)
print(d_test_b_ctx_coref_simplified.shape)

Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
(531835, 25)
(136259, 25)
(131893, 25)


In [15]:
process('ctx_coref', d_train_ctx_coref, d_test_a_ctx_coref, d_test_b_ctx_coref, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

/home/ymiao/.venvs/nerd/lib/python3.5/site-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


2019-11-21 18:35:11.162325 Start training
2019-11-21 18:45:46.857938 Training finished
2019-11-21 18:45:46.858688 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.960412059759452, corr_num: 17807
n_estimators: 4900, max_depth: 6, acc_validation: 0.8862450427885619, corr_num: 4246
n_estimators: 4900, max_depth: 6, acc_test: 0.841917502787068, corr_num: 3776
2019-11-21 18:45:53.258996 Evaluation finished
2019-11-21 18:45:53.330370 Model Saved: 4900_6_ctx_coref


In [24]:
process('test_local_time', d_train_ctx_coref, d_test_a_ctx_coref, d_test_b_ctx_coref, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-09-06 10:52:18.323442 Start training
2019-09-06 11:02:28.718053 Training finished
2019-09-06 11:02:28.718564 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.960412059759452, corr_num: 17807
n_estimators: 4900, max_depth: 6, acc_validation: 0.8862450427885619, corr_num: 4246
n_estimators: 4900, max_depth: 6, acc_test: 0.841917502787068, corr_num: 3776
2019-09-06 11:02:35.230482 Evaluation finished
2019-09-06 11:02:35.268297 Model Saved: 4900_6_test_local_time


In [32]:
model = load_model('4900_6_ctx_coref')
_data = trans_data(d_test_b_ctx_coref)
start_time = time.time()
model.predict(_data)
print(time.time() - start_time)

3.2189431190490723


In [92]:
process('ctx_coref_simplified', 
        d_train_ctx_coref_simplified, d_test_a_ctx_coref_simplified, d_test_b_ctx_coref_simplified, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

/home/ymiao/.venvs/nerd/lib/python3.5/site-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


2019-08-17 22:19:25.276378 Start training
2019-08-17 22:40:21.211973 Training finished
2019-08-17 22:40:21.212480 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9543174586052532, corr_num: 17694
n_estimators: 4900, max_depth: 6, acc_validation: 0.8753913587977458, corr_num: 4194
n_estimators: 4900, max_depth: 6, acc_test: 0.8345596432552954, corr_num: 3743
2019-08-17 22:40:28.108736 Evaluation finished
2019-08-17 22:40:28.153073 Model Saved: 4900_6_ctx_coref_simplified


In [35]:
process('ctx_coref', d_train_ctx_coref, d_test_a_ctx_coref, d_test_b_ctx_coref, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4400, 4900, 5400], max_depths=[5, 6, 7])

2019-08-14 22:32:57.221922 Start training
2019-08-14 22:47:21.100733 Training finished
2019-08-14 22:47:21.101321 Start evaluation
n_estimators: 4400, max_depth: 5, acc_training: 0.9417507146324362, corr_num: 17461
n_estimators: 4400, max_depth: 5, acc_validation: 0.87622625756627, corr_num: 4198
n_estimators: 4400, max_depth: 5, acc_test: 0.8434782608695652, corr_num: 3783
2019-08-14 22:47:26.760719 Evaluation finished
2019-08-14 22:47:26.783019 Model Saved: 4400_5_ctx_coref
2019-08-14 22:47:26.783118 Start training
2019-08-14 23:06:31.551324 Training finished
2019-08-14 23:06:31.551833 Start evaluation
n_estimators: 4400, max_depth: 6, acc_training: 0.9578232026320047, corr_num: 17759
n_estimators: 4400, max_depth: 6, acc_validation: 0.8860363180964308, corr_num: 4245
n_estimators: 4400, max_depth: 6, acc_test: 0.8439241917502787, corr_num: 3785
2019-08-14 23:06:38.254938 Evaluation finished
2019-08-14 23:06:38.291835 Model Saved: 4400_6_ctx_coref
2019-08-14 23:06:38.291945 Start tra

KeyboardInterrupt: 

In [16]:
def get_men_ents_tokens_idf_redis(men):
    ent_tok_dict = conn.hmget('idf:::50', men)[0]
    return json.loads(ent_tok_dict) if ent_tok_dict else {}

def get_men_ents_tokens_entropy_redis(men):
    ent_tok_dict = conn.hmget('entropy:::50', men)[0]
    return json.loads(ent_tok_dict) if ent_tok_dict else {}

def get_men_ents_tokens_tf_redis(men):
    ent_tok_dict = conn.hmget('tf:::50', men)[0]
    return json.loads(ent_tok_dict) if ent_tok_dict else {}

def get_men_tokens_redis(data_source, doc_id):
    if data_source == 'aida_conll':
        data_source = 'conll'
    tokens = conn.hmget('men-tok-ner-spacy-' + data_source, doc_id)[0]
    return json.loads(tokens) if tokens else {}

def fetch_mention_by_q_id(q_id):
    cur = conn_psql.cursor()
    sql = "SELECT aida_conll.surfaceform, aida_conll.doc_id FROM aida_conll WHERE aida_conll.id=%s AND annotation != 'NIL';"
    cur.execute(sql % q_id)
    row = cur.fetchone()
    cur.close()
    return row

def fetch_mention_by_q_id(q_id):
    cur = conn_psql.cursor()
    sql = "SELECT aida_conll.surfaceform, aida_conll.doc_id FROM aida_conll WHERE aida_conll.id=%s AND annotation != 'NIL';"
    cur.execute(sql % q_id)
    row = cur.fetchone()
    cur.close()
    return row

def fetch_mention_ent_by_q_id(q_id):
    cur = conn_psql.cursor()
    sql = "SELECT aida_conll.surfaceform, aida_conll.annotation, aida_conll.doc_id FROM aida_conll WHERE aida_conll.id=%s AND annotation != 'NIL';"
    cur.execute(sql % q_id)
    row = cur.fetchone()
    cur.close()
    return row

def fetch_mention_count_by_doc_id(q_id):
    cur = conn_psql.cursor()
    sql = "SELECT count(*) FROM aida_conll WHERE aida_conll.doc_id=%s AND annotation != 'NIL';"
    cur.execute(sql , (q_id,))
    row = cur.fetchone()
    cur.close()
    return row

def fetch_ents_by_doc_id(q_id):
    cur = conn_psql.cursor()
    sql = "SELECT annotation FROM aida_conll WHERE aida_conll.doc_id=%s AND annotation != 'NIL';"
    cur.execute(sql , (q_id,))
    row = cur.fetchall()
    cur.close()
    return row

def fetch_men_ents_by_doc_id(q_id):
    cur = conn_psql.cursor()
    sql = "SELECT id, annotation FROM aida_conll WHERE aida_conll.doc_id=%s AND annotation != 'NIL';"
    cur.execute(sql ,(q_id,))
    row = cur.fetchall()
    cur.close()
    return row

def fetch_q_ids_docs():
    cur = conn_psql.cursor()
    sql = "SELECT id, doc_id FROM aida_conll WHERE annotation != 'NIL';"
    cur.execute(sql)
    row = cur.fetchall()
    cur.close()
    return row

def fetch_ent_cates_by_ent_id(q_id):
    q_id = 'http://en.wikipedia.org/wiki/' + q_id
    cur = conn_psql.cursor()
    sql = "SELECT labels FROM dbpedia_ontology WHERE wikipedia_url=%s;"
    cur.execute(sql, (q_id,))
    row = cur.fetchone()
    cur.close()
    return row

def fetch_doc_ids_by_data_type(data_type):
    cur = conn_psql.cursor()
    sql = "SELECT distinct aida_conll.doc_id FROM aida_conll WHERE aida_conll.type=\'%s\';"
    cur.execute(sql % data_type)
    rows = cur.fetchall()
    cur.close()
    return rows

def fetch_doc_ids():
    cur = conn_psql.cursor()
    sql = "SELECT distinct aida_conll.doc_id FROM aida_conll;"
    cur.execute(sql)
    rows = cur.fetchall()
    cur.close()
    return rows


In [17]:
q_docs_dict = dict(fetch_q_ids_docs())

In [18]:
for k, v in q_docs_dict.items():
    print(k,'|', v)
    break

393217 | 863 Export


In [18]:
def true_labels(data_set):
    return data_set[:, -1].nonzero()[0].size

In [20]:
print(true_labels(d_train_ctx_coref))
print(true_labels(d_test_a_ctx_coref))
print(true_labels(d_test_b_ctx_coref))

18288
4699
4413


In [ ]:
print(true_labels(d_train_ctx_coref_can30))
print(true_labels(d_test_a_ctx_coref_can30))
print(true_labels(d_test_b_ctx_coref_can30))

In [97]:
model = load_model('4900_6_ctx_coref')
save_local_model_predictions(model, 
                             d_train_ctx_coref, d_train_ctx_raw, 
                             d_test_a_ctx_coref, d_test_a_ctx_raw,  
                             d_test_b_ctx_coref, d_test_b_ctx_raw,
                            top_k=3)

18288 4699 4413


In [93]:
model = load_model('4900_6_ctx_coref_simplified')
save_local_model_predictions(model, 
                             d_train_ctx_coref_simplified, d_train_ctx_raw, 
                             d_test_a_ctx_coref_simplified, d_test_a_ctx_raw,  
                             d_test_b_ctx_coref_simplified, d_test_b_ctx_raw,
                            top_k=3)

18288 4699 4413


Note: Generate the coherence features before running the following cells!

In [19]:
d_train_coh = fetch_all_features('basic_fea_coh', 'train', 'aida_conll')
d_test_a_coh = fetch_all_features('basic_fea_coh', 'testa', 'aida_conll')
d_test_b_coh = fetch_all_features('basic_fea_coh', 'testb', 'aida_conll')

In [20]:
print(d_train_coh.shape)
print(d_test_a_coh.shape)
print(d_test_b_coh.shape)

(531835, 24)
(136259, 24)
(131893, 24)


In [94]:
d_train_coh_simplified = fetch_all_features('basic_fea_coh_simplified', 'train', 'aida_conll')
d_test_a_coh_simplified = fetch_all_features('basic_fea_coh_simplified', 'testa', 'aida_conll')
d_test_b_coh_simplified = fetch_all_features('basic_fea_coh_simplified', 'testb', 'aida_conll')
print(d_train_coh_simplified.shape)
print(d_test_a_coh_simplified.shape)
print(d_test_b_coh_simplified.shape)

(531835, 24)
(136259, 24)
(131893, 24)


In [21]:
d_train_ctx_coref_coh  = combine_features(d_train_ctx_coref, d_train_coh)
d_test_a_ctx_coref_coh = combine_features(d_test_a_ctx_coref, d_test_a_coh)
d_test_b_ctx_coref_coh = combine_features(d_test_b_ctx_coref, d_test_b_coh)

Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...


In [22]:
print(d_train_ctx_coref_coh.shape)
print(d_test_a_ctx_coref_coh.shape)
print(d_test_b_ctx_coref_coh.shape)

(531835, 54)
(136259, 54)
(131893, 54)


In [95]:
d_train_ctx_coref_coh_simplified  = combine_features(d_train_ctx_coref_simplified, d_train_coh_simplified)
d_test_a_ctx_coref_coh_simplified = combine_features(d_test_a_ctx_coref_simplified, d_test_a_coh_simplified)
d_test_b_ctx_coref_coh_simplified = combine_features(d_test_b_ctx_coref_simplified, d_test_b_coh_simplified)
print(d_train_ctx_coref_coh_simplified.shape)
print(d_test_a_ctx_coref_coh_simplified.shape)
print(d_test_b_ctx_coref_coh_simplified.shape)

Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
(531835, 47)
(136259, 47)
(131893, 47)


In [24]:
process('ctx_coref_coh', d_train_ctx_coref_coh, d_test_a_ctx_coref_coh, d_test_b_ctx_coref_coh, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-16 16:15:29.913779 Start training
2019-08-16 16:58:15.664490 Training finished
2019-08-16 16:58:15.665003 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9752979882422739, corr_num: 18083
n_estimators: 4900, max_depth: 6, acc_validation: 0.8960551033187226, corr_num: 4293
n_estimators: 4900, max_depth: 6, acc_test: 0.89520624303233, corr_num: 4015
2019-08-16 16:58:23.727621 Evaluation finished
2019-08-16 16:58:23.775649 Model Saved: 4900_6_ctx_coref_coh


In [23]:
process('test_time', d_train_ctx_coref_coh, d_test_a_ctx_coref_coh, d_test_b_ctx_coref_coh, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

/home/ymiao/.venvs/nerd/lib/python3.5/site-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


2019-09-06 10:34:24.203613 Start training
2019-09-06 10:51:06.493577 Training finished
2019-09-06 10:51:06.494056 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9752979882422739, corr_num: 18083
n_estimators: 4900, max_depth: 6, acc_validation: 0.8960551033187226, corr_num: 4293
n_estimators: 4900, max_depth: 6, acc_test: 0.89520624303233, corr_num: 4015
2019-09-06 10:51:12.948869 Evaluation finished
2019-09-06 10:51:13.029273 Model Saved: 4900_6_test_time


In [33]:
for _ in range(2):
    process('test_time', d_train_ctx_coref_coh, d_test_a_ctx_coref_coh, d_test_b_ctx_coref_coh, 
            d_train_total, d_test_a_total, d_test_b_total,
            n_estimators=[4900], max_depths=[6])

2019-09-06 12:13:24.111611 Start training
2019-09-06 12:29:05.998538 Training finished
2019-09-06 12:29:05.999414 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9752979882422739, corr_num: 18083
n_estimators: 4900, max_depth: 6, acc_validation: 0.8960551033187226, corr_num: 4293
n_estimators: 4900, max_depth: 6, acc_test: 0.89520624303233, corr_num: 4015
2019-09-06 12:29:12.538808 Evaluation finished
2019-09-06 12:29:12.573252 Model Saved: 4900_6_test_time
2019-09-06 12:29:12.923037 Start training
2019-09-06 12:44:57.432827 Training finished
2019-09-06 12:44:57.433320 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9752979882422739, corr_num: 18083
n_estimators: 4900, max_depth: 6, acc_validation: 0.8960551033187226, corr_num: 4293
n_estimators: 4900, max_depth: 6, acc_test: 0.89520624303233, corr_num: 4015
2019-09-06 12:45:03.902554 Evaluation finished
2019-09-06 12:45:03.939429 Model Saved: 4900_6_test_time


In [96]:
process('ctx_coref_coh_simplified', 
        d_train_ctx_coref_coh_simplified, d_test_a_ctx_coref_coh_simplified, d_test_b_ctx_coref_coh_simplified, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-18 04:08:13.945035 Start training
2019-08-18 04:22:16.371638 Training finished
2019-08-18 04:22:16.372148 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.97508225014832, corr_num: 18079
n_estimators: 4900, max_depth: 6, acc_validation: 0.8956376539344605, corr_num: 4291
n_estimators: 4900, max_depth: 6, acc_test: 0.8945373467112597, corr_num: 4012
2019-08-18 04:22:22.936292 Evaluation finished
2019-08-18 04:22:22.976178 Model Saved: 4900_6_ctx_coref_coh_simplified


In [98]:
model = load_model('4900_6_ctx_coref_coh')
save_local_model_predictions(model, 
                             d_train_ctx_coref_coh, d_train_ctx_raw, 
                             d_test_a_ctx_coref_coh, d_test_a_ctx_raw,  
                             d_test_b_ctx_coref_coh, d_test_b_ctx_raw)

18288 4699 4413


In [27]:
d_train_ctx_coh  = combine_features(d_train_ctx, d_train_coh)
d_test_a_ctx_coh = combine_features(d_test_a_ctx, d_test_a_coh)
d_test_b_ctx_coh = combine_features(d_test_b_ctx, d_test_b_coh)

Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...


In [28]:
print(d_train_ctx_coh.shape)
print(d_test_a_ctx_coh.shape)
print(d_test_b_ctx_coh.shape)

(531835, 43)
(136259, 43)
(131893, 43)


In [29]:
process('ctx_coh', d_train_ctx_coh, d_test_a_ctx_coh, d_test_b_ctx_coh, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-14 21:31:37.349964 Start training
2019-08-14 21:52:37.433294 Training finished
2019-08-14 21:52:37.433794 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9744350358664581, corr_num: 18067
n_estimators: 4900, max_depth: 6, acc_validation: 0.8956376539344605, corr_num: 4291
n_estimators: 4900, max_depth: 6, acc_test: 0.8869565217391304, corr_num: 3978
2019-08-14 21:52:45.750054 Evaluation finished
2019-08-14 21:52:45.805173 Model Saved: 4900_6_ctx_coh


In [30]:
d_train_coref_coh  = combine_features(d_train_coref, d_train_coh)
d_test_a_coref_coh = combine_features(d_test_a_coref, d_test_a_coh)
d_test_b_coref_coh = combine_features(d_test_b_coref, d_test_b_coh)

Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...


In [31]:
print(d_train_coref_coh.shape)
print(d_test_a_coref_coh.shape)
print(d_test_b_coref_coh.shape)

(531835, 35)
(136259, 35)
(131893, 35)


In [32]:
process('coref_coh', d_train_coref_coh, d_test_a_coref_coh, d_test_b_coref_coh, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-14 21:52:47.628595 Start training
2019-08-14 22:07:56.546286 Training finished
2019-08-14 22:07:56.547230 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9729248692087805, corr_num: 18039
n_estimators: 4900, max_depth: 6, acc_validation: 0.891880609476101, corr_num: 4273
n_estimators: 4900, max_depth: 6, acc_test: 0.8896321070234113, corr_num: 3990
2019-08-14 22:08:03.104239 Evaluation finished
2019-08-14 22:08:03.141590 Model Saved: 4900_6_coref_coh


In [33]:
process('coref', d_train_coref, d_test_a_coref, d_test_b_coref, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-14 22:08:03.231453 Start training
2019-08-14 22:14:20.120110 Training finished
2019-08-14 22:14:20.120623 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.915700339787498, corr_num: 16978
n_estimators: 4900, max_depth: 6, acc_validation: 0.8509705698184096, corr_num: 4077
n_estimators: 4900, max_depth: 6, acc_test: 0.7991081382385731, corr_num: 3584
2019-08-14 22:14:26.610881 Evaluation finished
2019-08-14 22:14:26.646472 Model Saved: 4900_6_coref


In [34]:
process('coh', d_train_coh, d_test_a_coh, d_test_b_coh, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-14 22:14:26.814485 Start training
2019-08-14 22:24:18.044900 Training finished
2019-08-14 22:24:18.045400 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.8987109648886252, corr_num: 16663
n_estimators: 4900, max_depth: 6, acc_validation: 0.7845961177207263, corr_num: 3759
n_estimators: 4900, max_depth: 6, acc_test: 0.7670011148272018, corr_num: 3440
2019-08-14 22:24:24.821150 Evaluation finished
2019-08-14 22:24:24.867810 Model Saved: 4900_6_coh


In [25]:
d_train_ctx_can30 = fetch_all_features('fea_ctx_can30', 'train', 'aida_conll')
d_test_a_ctx_can30 = fetch_all_features('fea_ctx_can30', 'testa', 'aida_conll')
d_test_b_ctx_can30 = fetch_all_features('fea_ctx_can30', 'testb', 'aida_conll')
print(d_train_ctx_can30.shape)
print(d_test_a_ctx_can30.shape)
print(d_test_b_ctx_can30.shape)

(342796, 21)
(86537, 21)
(84698, 21)


In [26]:
process('ctx_can30', d_train_ctx_can30, d_test_a_ctx_can30, d_test_b_ctx_can30, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-16 17:57:18.492319 Start training
2019-08-16 18:05:53.209039 Training finished
2019-08-16 18:05:53.209538 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9486543336389623, corr_num: 17589
n_estimators: 4900, max_depth: 6, acc_validation: 0.8630766019620121, corr_num: 4135
n_estimators: 4900, max_depth: 6, acc_test: 0.8303232998885173, corr_num: 3724
2019-08-16 18:05:58.269448 Evaluation finished
2019-08-16 18:05:58.331593 Model Saved: 4900_6_ctx_can30


In [27]:
d_train_coref_can30 = fetch_all_features('fea_coref_can30', 'train', 'aida_conll')
d_test_a_coref_can30 = fetch_all_features('fea_coref_can30', 'testa', 'aida_conll')
d_test_b_coref_can30 = fetch_all_features('fea_coref_can30', 'testb', 'aida_conll')
print(d_train_coref_can30.shape)
print(d_test_a_coref_can30.shape)
print(d_test_b_coref_can30.shape)

(342796, 13)
(86537, 13)
(84698, 13)


In [28]:
process('coref_can30', d_train_coref_can30, d_test_a_coref_can30, d_test_b_coref_can30, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-16 18:06:07.840280 Start training
2019-08-16 18:12:44.434335 Training finished
2019-08-16 18:12:44.435234 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9165093576398252, corr_num: 16993
n_estimators: 4900, max_depth: 6, acc_validation: 0.8474222500521812, corr_num: 4060
n_estimators: 4900, max_depth: 6, acc_test: 0.8028985507246377, corr_num: 3601
2019-08-16 18:12:49.645264 Evaluation finished
2019-08-16 18:12:49.666220 Model Saved: 4900_6_coref_can30


In [29]:
d_train_ctx_coref_can30  = combine_features(d_train_ctx_can30, d_train_ctx_can30)
d_test_a_ctx_coref_can30 = combine_features(d_test_a_ctx_can30, d_test_a_ctx_can30)
d_test_b_ctx_coref_can30 = combine_features(d_test_b_ctx_can30, d_test_b_ctx_can30)
print(d_train_ctx_coref_can30.shape)
print(d_test_a_ctx_coref_can30.shape)
print(d_test_b_ctx_coref_can30.shape)

Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
(342796, 40)
(86537, 40)
(84698, 40)


In [58]:
process('ctx_coref_can30', d_train_ctx_coref_can30, d_test_a_ctx_coref_can30, d_test_b_ctx_coref_can30, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-17 12:11:52.627563 Start training
2019-08-17 12:20:02.396669 Training finished
2019-08-17 12:20:02.397178 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9486543336389623, corr_num: 17589
n_estimators: 4900, max_depth: 6, acc_validation: 0.8630766019620121, corr_num: 4135
n_estimators: 4900, max_depth: 6, acc_test: 0.8303232998885173, corr_num: 3724
2019-08-17 12:20:06.587044 Evaluation finished
2019-08-17 12:20:06.637740 Model Saved: 4900_6_ctx_coref_can30


In [27]:
model = load_model('4900_6_ctx_coref')
save_local_model_predictions(model, 
                             d_train_ctx_coref, d_train_ctx_raw, 
                             d_test_a_ctx_coref, d_test_a_ctx_raw,  
                             d_test_b_ctx_coref, d_test_b_ctx_raw, top_k=3)

test_a_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testa'))
test_b_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testb'))
print('test_a_filter', len(test_a_filter))
print('test_b_filter', len(test_b_filter))

process('ctx_coref', d_train_ctx_coref, d_test_a_ctx_coref, d_test_b_ctx_coref, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6], train_mode=False, 
        test_a_filter=test_a_filter, test_b_filter=test_b_filter)

Number of groups: 18288 4699 4413
test_a_filter 12008
test_b_filter 11383
2019-08-22 05:55:10.298464 Loading model: 4900_6_ctx_coref
2019-08-22 05:55:10.312582 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.960412059759452, corr_num: 17807
n_estimators: 4900, max_depth: 6, acc_validation: 0.9194322688374035, corr_num: 4405
n_estimators: 4900, max_depth: 6, acc_test: 0.87603121516165, corr_num: 3929
2019-08-22 05:55:23.910699 Evaluation finished


In [25]:
model = load_model('4900_6_ctx_coref')
save_local_model_predictions(model, 
                             d_train_ctx_coref, d_train_ctx_raw, 
                             d_test_a_ctx_coref, d_test_a_ctx_raw,  
                             d_test_b_ctx_coref, d_test_b_ctx_raw, top_k=3)

test_a_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testa'))
test_b_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testb'))
print('test_a_filter', len(test_a_filter))
print('test_b_filter', len(test_b_filter))

process('ctx_coref_coh', d_train_ctx_coref_coh, d_test_a_ctx_coref_coh, d_test_b_ctx_coref_coh, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6], train_mode=False, 
        test_a_filter=test_a_filter, test_b_filter=test_b_filter)

/home/ymiao/.venvs/nerd/lib/python3.5/site-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


Number of groups: 18288 4699 4413
test_a_filter 12008
test_b_filter 11383
2019-08-25 21:33:32.938384 Loading model: 4900_6_ctx_coref_coh
2019-08-25 21:33:32.983139 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9752979882422739, corr_num: 18083
n_estimators: 4900, max_depth: 6, acc_validation: 0.9160926737633062, corr_num: 4389
n_estimators: 4900, max_depth: 6, acc_test: 0.89520624303233, corr_num: 4015
2019-08-25 21:33:45.693810 Evaluation finished


In [26]:
model = load_model('4900_6_ctx_coref')
save_local_model_predictions(model, 
                             d_train_ctx_coref, d_train_ctx_raw, 
                             d_test_a_ctx_coref, d_test_a_ctx_raw,  
                             d_test_b_ctx_coref, d_test_b_ctx_raw, top_k=5)

test_a_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testa'))
test_b_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testb'))
print('test_a_filter', len(test_a_filter))
print('test_b_filter', len(test_b_filter))

process('ctx_coref_coh', d_train_ctx_coref_coh, d_test_a_ctx_coref_coh, d_test_b_ctx_coref_coh, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6], train_mode=False, 
        test_a_filter=test_a_filter, test_b_filter=test_b_filter)

Number of groups: 18288 4699 4413
test_a_filter 18773
test_b_filter 17912
2019-08-25 21:34:16.567208 Loading model: 4900_6_ctx_coref_coh
2019-08-25 21:34:16.583254 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9752979882422739, corr_num: 18083
n_estimators: 4900, max_depth: 6, acc_validation: 0.9198497182216656, corr_num: 4407
n_estimators: 4900, max_depth: 6, acc_test: 0.9110367892976589, corr_num: 4086
2019-08-25 21:34:30.088704 Evaluation finished


In [29]:
model = load_model('4900_6_ctx_coref_coh')
save_local_model_predictions(model, 
                             d_train_ctx_coref_coh, d_train_ctx_raw, 
                             d_test_a_ctx_coref_coh, d_test_a_ctx_raw,  
                             d_test_b_ctx_coref_coh, d_test_b_ctx_raw, top_k=3)

test_a_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testa'))
test_b_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testb'))
print('test_a_filter', len(test_a_filter))
print('test_b_filter', len(test_b_filter))

process('ctx_coref_coh', d_train_ctx_coref_coh, d_test_a_ctx_coref_coh, d_test_b_ctx_coref_coh, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6], train_mode=False, 
        test_a_filter=test_a_filter, test_b_filter=test_b_filter)

Number of groups: 18288 4699 4413
test_a_filter 12008
test_b_filter 11383
2019-08-22 05:59:59.075727 Loading model: 4900_6_ctx_coref_coh
2019-08-22 05:59:59.091601 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9752979882422739, corr_num: 18083
n_estimators: 4900, max_depth: 6, acc_validation: 0.9375913170528073, corr_num: 4492
n_estimators: 4900, max_depth: 6, acc_test: 0.9311036789297659, corr_num: 4176
2019-08-22 06:00:13.092215 Evaluation finished


In [49]:
from collections import defaultdict
def evalerror_detail_log_xl(preds, dt, d_tal_size):
    top2_correct = 0
    top3_correct = 0
    correct_top2_diff_total = 0
    wrong_top2_diff_total = 0
    correct_has_top2 = 0
    wrong_has_top2 = 0
    
    
    d_l = dt.get_label()
    idxs = np.where(d_l == 1)[0]
    d_groups = np.append(np.delete(idxs, 0), len(d_l)) - idxs
    correct_results = {}
    wrong_results = {}
    duplicates_results = {}
    group_info = {}
    matched_ids = []
    q_id = 0
    for x in d_groups:
        #print("For group {}, id = {}".format(x, q_id+x))
        pre_res = preds[q_id: x + q_id]
        
        sorted_pre_res = np.sort(pre_res)
        reverse_res = sorted_pre_res[::-1]
        #print(reverse_res[0:3])
        pre_res = reverse_res
        if (max(pre_res) != pre_res[0]):
            print("ERROR")
            break
        
        if(len(pre_res) == 1 or preds[q_id] == pre_res[0] or preds[q_id] == pre_res[1]):
                top2_correct += 1
                top3_correct += 1
        else:   
            if(len(pre_res) == 2 or preds[q_id] == pre_res[2]):
                top3_correct += 1

        
        if(preds[q_id] == max(pre_res)):
            correct_results[q_id] = pre_res
            
            if len([x for x in pre_res if x == preds[q_id]]) == 1:
                matched_ids.append(q_id)
                
                if (len(pre_res) > 1):
                    correct_top2_diff_total += ( pre_res[0] - pre_res[1]) 
                    correct_has_top2 +=1
                    
            else:
                duplicates_results[q_id] = pre_res
        else:
            wrong_results[q_id] = pre_res
            if (len(pre_res) > 1):
                wrong_top2_diff_total += ( pre_res[0] - pre_res[1])
                wrong_has_top2 += 1
                
            #print (pre_res[0:3])
        q_id += x
    precision = float(len(matched_ids)) / len(d_groups)
    recall = float(len(matched_ids)) / d_tal_size
    f1 = 2 * precision * recall / (precision + recall)
    top2_acc = top2_correct / d_tal_size
    top3_acc = top3_correct / d_tal_size
    print (top2_correct, top3_correct, top2_acc, top3_acc)
    
    correct_top2_diff_avg = correct_top2_diff_total / correct_has_top2
    wrong_top2_diff_avg = wrong_top2_diff_total / wrong_has_top2
    print (correct_top2_diff_avg, wrong_top2_diff_avg)
    
    return len(matched_ids), precision, recall, f1, correct_results, wrong_results, duplicates_results

In [72]:
process('ctx_coref', d_train_ctx_coref, d_test_a_ctx_coref, d_test_b_ctx_coref, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6], train_mode=False, eval_func=evalerror_detail_log_xl)

2019-08-17 15:00:28.566473 Loading model: 4900_6_ctx_coref
2019-08-17 15:00:28.592544 Start evaluation
18247 18282 0.9841432500943854 0.9860309584164824
5.024404213151237 0.7807343878031299
n_estimators: 4900, max_depth: 6, acc_training: 0.960412059759452, corr_num: 17807
4557 4624 0.9511584220413275 0.9651429764141098
4.909797580852987 2.0565747870028153
n_estimators: 4900, max_depth: 6, acc_validation: 0.8862450427885619, corr_num: 4246
4084 4248 0.9105908584169454 0.9471571906354515
4.883615036342713 1.7138141848241226
n_estimators: 4900, max_depth: 6, acc_test: 0.841917502787068, corr_num: 3776
2019-08-17 15:00:47.929463 Evaluation finished


In [27]:
d_train_coh_global = fetch_all_features('basic_fea_coh_global', 'train', 'aida_conll')
d_test_a_coh_global = fetch_all_features('basic_fea_coh_global', 'testa', 'aida_conll')
d_test_b_coh_global = fetch_all_features('basic_fea_coh_global', 'testb', 'aida_conll')
print(d_train_coh_global.shape)
print(d_test_a_coh_global.shape)
print(d_test_b_coh_global.shape)

(531835, 24)
(136259, 24)
(131893, 24)


In [28]:
d_train_ctx_coref_coh_global  = combine_features(d_train_ctx_coref, d_train_coh_global)
d_test_a_ctx_coref_coh_global = combine_features(d_test_a_ctx_coref, d_test_a_coh_global)
d_test_b_ctx_coref_coh_global = combine_features(d_test_b_ctx_coref, d_test_b_coh_global)
print(d_train_ctx_coref_coh_global.shape)
print(d_test_a_ctx_coref_coh_global.shape)
print(d_test_b_ctx_coref_coh_global.shape)

Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
(531835, 54)
(136259, 54)
(131893, 54)


In [101]:
process('ctx_coref_coh_global', 
        d_train_ctx_coref_coh_global, d_test_a_ctx_coref_coh_global, d_test_b_ctx_coref_coh_global, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-18 13:48:28.193368 Start training
2019-08-18 14:04:14.859434 Training finished
2019-08-18 14:04:14.859934 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9756215953832048, corr_num: 18089
n_estimators: 4900, max_depth: 6, acc_validation: 0.8979336255479023, corr_num: 4302
n_estimators: 4900, max_depth: 6, acc_test: 0.9016722408026756, corr_num: 4044
2019-08-18 14:04:21.287457 Evaluation finished
2019-08-18 14:04:21.318611 Model Saved: 4900_6_ctx_coref_coh_global


In [29]:
model = load_model('4900_6_ctx_coref_coh')
save_local_model_predictions(model, 
                             d_train_ctx_coref_coh, d_train_ctx_raw, 
                             d_test_a_ctx_coref_coh, d_test_a_ctx_raw,  
                             d_test_b_ctx_coref_coh, d_test_b_ctx_raw, top_k=3)

test_a_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testa'))
test_b_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testb'))
print('test_a_filter', len(test_a_filter))
print('test_b_filter', len(test_b_filter))

process('ctx_coref_coh_global', 
        d_train_ctx_coref_coh_global, d_test_a_ctx_coref_coh_global, d_test_b_ctx_coref_coh_global, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6], train_mode=False, 
        test_a_filter=test_a_filter, test_b_filter=test_b_filter)

Number of groups: 18288 4699 4413
test_a_filter 12008
test_b_filter 11383
2019-08-26 00:53:00.313850 Loading model: 4900_6_ctx_coref_coh_global
2019-08-26 00:53:00.357894 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9756215953832048, corr_num: 18089
n_estimators: 4900, max_depth: 6, acc_validation: 0.9315383009810061, corr_num: 4463
n_estimators: 4900, max_depth: 6, acc_test: 0.9286510590858417, corr_num: 4165
2019-08-26 00:53:13.537194 Evaluation finished


In [30]:
model = load_model('4900_6_ctx_coref_coh')
save_local_model_predictions(model, 
                             d_train_ctx_coref_coh, d_train_ctx_raw, 
                             d_test_a_ctx_coref_coh, d_test_a_ctx_raw,  
                             d_test_b_ctx_coref_coh, d_test_b_ctx_raw, top_k=5)

test_a_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testa'))
test_b_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testb'))
print('test_a_filter', len(test_a_filter))
print('test_b_filter', len(test_b_filter))

process('ctx_coref_coh_global', 
        d_train_ctx_coref_coh_global, d_test_a_ctx_coref_coh_global, d_test_b_ctx_coref_coh_global, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6], train_mode=False, 
        test_a_filter=test_a_filter, test_b_filter=test_b_filter)

Number of groups: 18288 4699 4413
test_a_filter 18773
test_b_filter 17912
2019-08-26 00:55:07.879648 Loading model: 4900_6_ctx_coref_coh_global
2019-08-26 00:55:07.895017 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9756215953832048, corr_num: 18089
n_estimators: 4900, max_depth: 6, acc_validation: 0.9361302442078898, corr_num: 4485
n_estimators: 4900, max_depth: 6, acc_test: 0.9313266443701226, corr_num: 4177
2019-08-26 00:55:20.913263 Evaluation finished


In [129]:
model = load_model('4900_6_ctx_coref_coh_global')
save_local_model_predictions(model, 
                             d_train_ctx_coref_coh_global, d_train_ctx_raw, 
                             d_test_a_ctx_coref_coh_global, d_test_a_ctx_raw,  
                             d_test_b_ctx_coref_coh_global, d_test_b_ctx_raw)

18288 4699 4413


In [83]:
d_train_coh_global2 = fetch_all_features('basic_fea_coh_global2', 'train', 'aida_conll')
d_test_a_coh_global2 = fetch_all_features('basic_fea_coh_global2', 'testa', 'aida_conll')
d_test_b_coh_global2 = fetch_all_features('basic_fea_coh_global2', 'testb', 'aida_conll')
print(d_train_coh_global2.shape)
print(d_test_a_coh_global2.shape)
print(d_test_b_coh_global2.shape)

(531835, 24)
(136259, 24)
(131893, 24)


In [84]:
d_train_ctx_coref_coh_global2  = combine_features(d_train_ctx_coref, d_train_coh_global2)
d_test_a_ctx_coref_coh_global2 = combine_features(d_test_a_ctx_coref, d_test_a_coh_global2)
d_test_b_ctx_coref_coh_global2 = combine_features(d_test_b_ctx_coref, d_test_b_coh_global2)
print(d_train_ctx_coref_coh_global2.shape)
print(d_test_a_ctx_coref_coh_global2.shape)
print(d_test_b_ctx_coref_coh_global2.shape)

Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
(531835, 54)
(136259, 54)
(131893, 54)


In [125]:
process('ctx_coref_coh_global2', 
        d_train_ctx_coref_coh_global2, d_test_a_ctx_coref_coh_global2, d_test_b_ctx_coref_coh_global2, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

/home/ymiao/.venvs/nerd/lib/python3.5/site-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


2019-08-18 17:15:53.223591 Start training
2019-08-18 17:39:37.925763 Training finished
2019-08-18 17:39:37.926259 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9754597918127393, corr_num: 18086
n_estimators: 4900, max_depth: 6, acc_validation: 0.8966812773951158, corr_num: 4296
n_estimators: 4900, max_depth: 6, acc_test: 0.9036789297658863, corr_num: 4053
2019-08-18 17:39:45.818873 Evaluation finished
2019-08-18 17:39:45.850015 Model Saved: 4900_6_ctx_coref_coh_global2


In [126]:
process('ctx_coref_coh_global2', 
        d_train_ctx_coref_coh_global2, d_test_a_ctx_coref_coh_global2, d_test_b_ctx_coref_coh_global2, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6], train_mode=False, eval_func=evalerror_detail_log_xl)

/home/ymiao/.venvs/nerd/lib/python3.5/site-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


2019-08-18 18:30:56.291661 Loading model: 4900_6_ctx_coref_coh_global2
2019-08-18 18:30:56.309111 Start evaluation
18283 18288 0.9860848929399709 0.9863545655574133
6.5721247553232995 0.5639766824714253
n_estimators: 4900, max_depth: 6, acc_training: 0.9754597918127393, corr_num: 18086
4585 4639 0.9570027134209977 0.968273846796076
6.396990317589336 2.4666858658532353
n_estimators: 4900, max_depth: 6, acc_validation: 0.8966812773951158, corr_num: 4296
4323 4377 0.9638795986622074 0.9759197324414716
6.318367755398238 2.287468268334026
n_estimators: 4900, max_depth: 6, acc_test: 0.9036789297658863, corr_num: 4053
2019-08-18 18:31:16.750204 Evaluation finished


In [130]:
model = load_model('4900_6_ctx_coref_coh_global')
save_local_model_predictions(model, 
                             d_train_ctx_coref_coh_global, d_train_ctx_raw, 
                             d_test_a_ctx_coref_coh_global, d_test_a_ctx_raw,  
                             d_test_b_ctx_coref_coh_global, d_test_b_ctx_raw, top_k=3)

test_a_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testa'))
test_b_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testb'))
print('test_a_filter', len(test_a_filter))
print('test_b_filter', len(test_b_filter))

process('ctx_coref_coh_global2', 
        d_train_ctx_coref_coh_global2, d_test_a_ctx_coref_coh_global2, d_test_b_ctx_coref_coh_global2, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6], train_mode=False, 
        test_a_filter=test_a_filter, test_b_filter=test_b_filter)

18288 4699 4413
test_a_filter 12008
test_b_filter 11383
2019-08-18 18:37:44.108521 Loading model: 4900_6_ctx_coref_coh_global2
2019-08-18 18:37:44.129362 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9754597918127393, corr_num: 18086
n_estimators: 4900, max_depth: 6, acc_validation: 0.9307034022124817, corr_num: 4459
n_estimators: 4900, max_depth: 6, acc_test: 0.9328874024526198, corr_num: 4184
2019-08-18 18:37:59.564449 Evaluation finished


In [133]:
model = load_model('4900_6_ctx_coref_coh_global')
save_local_model_predictions(model, 
                             d_train_ctx_coref_coh_global, d_train_ctx_raw, 
                             d_test_a_ctx_coref_coh_global, d_test_a_ctx_raw,  
                             d_test_b_ctx_coref_coh_global, d_test_b_ctx_raw, top_k=5)

test_a_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testa'))
test_b_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testb'))
print('test_a_filter', len(test_a_filter))
print('test_b_filter', len(test_b_filter))

process('ctx_coref_coh_global2', 
        d_train_ctx_coref_coh_global2, d_test_a_ctx_coref_coh_global2, d_test_b_ctx_coref_coh_global2, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6], train_mode=False, 
        test_a_filter=test_a_filter, test_b_filter=test_b_filter)

Number of groups: 18288 4699 4413
test_a_filter 18773
test_b_filter 17912
2019-08-18 18:42:04.567337 Loading model: 4900_6_ctx_coref_coh_global2
2019-08-18 18:42:04.584313 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9754597918127393, corr_num: 18086
n_estimators: 4900, max_depth: 6, acc_validation: 0.9352953454393654, corr_num: 4481
n_estimators: 4900, max_depth: 6, acc_test: 0.9348940914158306, corr_num: 4193
2019-08-18 18:42:21.044766 Evaluation finished


In [134]:
model = load_model('4900_6_ctx_coref_coh_global2')
save_local_model_predictions(model, 
                             d_train_ctx_coref_coh_global2, d_train_ctx_raw, 
                             d_test_a_ctx_coref_coh_global2, d_test_a_ctx_raw,  
                             d_test_b_ctx_coref_coh_global2, d_test_b_ctx_raw)

Number of groups: 18288 4699 4413


In [85]:
d_train_coh_global3 = fetch_all_features('basic_fea_coh_global3', 'train', 'aida_conll')
d_test_a_coh_global3 = fetch_all_features('basic_fea_coh_global3', 'testa', 'aida_conll')
d_test_b_coh_global3 = fetch_all_features('basic_fea_coh_global3', 'testb', 'aida_conll')
print(d_train_coh_global3.shape)
print(d_test_a_coh_global3.shape)
print(d_test_b_coh_global3.shape)

(531835, 24)
(136259, 24)
(131893, 24)


In [86]:
d_train_ctx_coref_coh_global3  = combine_features(d_train_ctx_coref, d_train_coh_global3)
d_test_a_ctx_coref_coh_global3 = combine_features(d_test_a_ctx_coref, d_test_a_coh_global3)
d_test_b_ctx_coref_coh_global3 = combine_features(d_test_b_ctx_coref, d_test_b_coh_global3)
print(d_train_ctx_coref_coh_global3.shape)
print(d_test_a_ctx_coref_coh_global3.shape)
print(d_test_b_ctx_coref_coh_global3.shape)

Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
(531835, 54)
(136259, 54)
(131893, 54)


In [137]:
process('ctx_coref_coh_global3', 
        d_train_ctx_coref_coh_global3, d_test_a_ctx_coref_coh_global3, d_test_b_ctx_coref_coh_global3, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-18 21:03:12.099376 Start training
2019-08-18 21:23:23.152517 Training finished
2019-08-18 21:23:23.153027 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9755676608597164, corr_num: 18088
n_estimators: 4900, max_depth: 6, acc_validation: 0.897307451471509, corr_num: 4299
n_estimators: 4900, max_depth: 6, acc_test: 0.9083612040133779, corr_num: 4074
2019-08-18 21:23:30.953078 Evaluation finished
2019-08-18 21:23:30.983750 Model Saved: 4900_6_ctx_coref_coh_global3


In [139]:
model = load_model('4900_6_ctx_coref_coh_global2')
save_local_model_predictions(model, 
                             d_train_ctx_coref_coh_global2, d_train_ctx_raw, 
                             d_test_a_ctx_coref_coh_global2, d_test_a_ctx_raw,  
                             d_test_b_ctx_coref_coh_global2, d_test_b_ctx_raw, top_k=3)

test_a_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testa'))
test_b_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testb'))
print('test_a_filter', len(test_a_filter))
print('test_b_filter', len(test_b_filter))

process('ctx_coref_coh_global3', 
        d_train_ctx_coref_coh_global3, d_test_a_ctx_coref_coh_global3, d_test_b_ctx_coref_coh_global3, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6], train_mode=False, 
        test_a_filter=test_a_filter, test_b_filter=test_b_filter)

Number of groups: 18288 4699 4413
test_a_filter 12008
test_b_filter 11383
2019-08-18 21:47:41.418188 Loading model: 4900_6_ctx_coref_coh_global3
2019-08-18 21:47:41.435355 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9755676608597164, corr_num: 18088
n_estimators: 4900, max_depth: 6, acc_validation: 0.9323731997495304, corr_num: 4467
n_estimators: 4900, max_depth: 6, acc_test: 0.9369007803790412, corr_num: 4202
2019-08-18 21:47:56.788129 Evaluation finished


In [138]:
model = load_model('4900_6_ctx_coref_coh_global2')
save_local_model_predictions(model, 
                             d_train_ctx_coref_coh_global2, d_train_ctx_raw, 
                             d_test_a_ctx_coref_coh_global2, d_test_a_ctx_raw,  
                             d_test_b_ctx_coref_coh_global2, d_test_b_ctx_raw, top_k=5)

test_a_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testa'))
test_b_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testb'))
print('test_a_filter', len(test_a_filter))
print('test_b_filter', len(test_b_filter))

process('ctx_coref_coh_global3', 
        d_train_ctx_coref_coh_global3, d_test_a_ctx_coref_coh_global3, d_test_b_ctx_coref_coh_global3, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6], train_mode=False, 
        test_a_filter=test_a_filter, test_b_filter=test_b_filter)

Number of groups: 18288 4699 4413
test_a_filter 18773
test_b_filter 17912
2019-08-18 21:45:48.193852 Loading model: 4900_6_ctx_coref_coh_global3
2019-08-18 21:45:48.211063 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9755676608597164, corr_num: 18088
n_estimators: 4900, max_depth: 6, acc_validation: 0.9380087664370695, corr_num: 4494
n_estimators: 4900, max_depth: 6, acc_test: 0.9380156075808249, corr_num: 4207
2019-08-18 21:46:04.691186 Evaluation finished


In [140]:
model = load_model('4900_6_ctx_coref_coh_global3')
save_local_model_predictions(model, 
                             d_train_ctx_coref_coh_global3, d_train_ctx_raw, 
                             d_test_a_ctx_coref_coh_global3, d_test_a_ctx_raw,  
                             d_test_b_ctx_coref_coh_global3, d_test_b_ctx_raw)

Number of groups: 18288 4699 4413


In [87]:
d_train_coh_global4 = fetch_all_features('basic_fea_coh_global4', 'train', 'aida_conll')
d_test_a_coh_global4 = fetch_all_features('basic_fea_coh_global4', 'testa', 'aida_conll')
d_test_b_coh_global4 = fetch_all_features('basic_fea_coh_global4', 'testb', 'aida_conll')
print(d_train_coh_global4.shape)
print(d_test_a_coh_global4.shape)
print(d_test_b_coh_global4.shape)

(531835, 24)
(136259, 24)
(131893, 24)


In [88]:
d_train_ctx_coref_coh_global4  = combine_features(d_train_ctx_coref, d_train_coh_global4)
d_test_a_ctx_coref_coh_global4 = combine_features(d_test_a_ctx_coref, d_test_a_coh_global4)
d_test_b_ctx_coref_coh_global4 = combine_features(d_test_b_ctx_coref, d_test_b_coh_global4)
print(d_train_ctx_coref_coh_global4.shape)
print(d_test_a_ctx_coref_coh_global4.shape)
print(d_test_b_ctx_coref_coh_global4.shape)

Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
(531835, 54)
(136259, 54)
(131893, 54)


In [143]:
process('ctx_coref_coh_global4', 
        d_train_ctx_coref_coh_global4, d_test_a_ctx_coref_coh_global4, d_test_b_ctx_coref_coh_global4, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-19 00:38:00.343219 Start training
2019-08-19 00:59:38.845629 Training finished
2019-08-19 00:59:38.846134 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9755676608597164, corr_num: 18088
n_estimators: 4900, max_depth: 6, acc_validation: 0.8964725527029848, corr_num: 4295
n_estimators: 4900, max_depth: 6, acc_test: 0.9059085841694537, corr_num: 4063
2019-08-19 00:59:45.734847 Evaluation finished
2019-08-19 00:59:45.767034 Model Saved: 4900_6_ctx_coref_coh_global4


In [36]:
model = load_model('4900_6_ctx_coref_coh_global3')
save_local_model_predictions(model, 
                             d_train_ctx_coref_coh_global3, d_train_ctx_raw, 
                             d_test_a_ctx_coref_coh_global3, d_test_a_ctx_raw,  
                             d_test_b_ctx_coref_coh_global3, d_test_b_ctx_raw, top_k=3)

test_a_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testa'))
test_b_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testb'))
print('test_a_filter', len(test_a_filter))
print('test_b_filter', len(test_b_filter))

process('ctx_coref_coh_global4', 
        d_train_ctx_coref_coh_global4, d_test_a_ctx_coref_coh_global4, d_test_b_ctx_coref_coh_global4, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6], train_mode=False, 
        test_a_filter=test_a_filter, test_b_filter=test_b_filter)

Number of groups: 18288 4699 4413
test_a_filter 12008
test_b_filter 11383
2019-08-26 01:03:54.222079 Loading model: 4900_6_ctx_coref_coh_global4
2019-08-26 01:03:54.265395 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9755676608597164, corr_num: 18088
n_estimators: 4900, max_depth: 6, acc_validation: 0.9315383009810061, corr_num: 4463
n_estimators: 4900, max_depth: 6, acc_test: 0.9348940914158306, corr_num: 4193
2019-08-26 01:04:07.103186 Evaluation finished


In [37]:
model = load_model('4900_6_ctx_coref_coh_global3')
save_local_model_predictions(model, 
                             d_train_ctx_coref_coh_global3, d_train_ctx_raw, 
                             d_test_a_ctx_coref_coh_global3, d_test_a_ctx_raw,  
                             d_test_b_ctx_coref_coh_global3, d_test_b_ctx_raw, top_k=5)

test_a_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testa'))
test_b_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testb'))
print('test_a_filter', len(test_a_filter))
print('test_b_filter', len(test_b_filter))

process('ctx_coref_coh_global4', 
        d_train_ctx_coref_coh_global4, d_test_a_ctx_coref_coh_global4, d_test_b_ctx_coref_coh_global4, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6], train_mode=False, 
        test_a_filter=test_a_filter, test_b_filter=test_b_filter)

Number of groups: 18288 4699 4413
test_a_filter 18773
test_b_filter 17912
2019-08-26 01:04:45.244714 Loading model: 4900_6_ctx_coref_coh_global4
2019-08-26 01:04:45.259814 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9755676608597164, corr_num: 18088
n_estimators: 4900, max_depth: 6, acc_validation: 0.9359215195157587, corr_num: 4484
n_estimators: 4900, max_depth: 6, acc_test: 0.937123745819398, corr_num: 4203
2019-08-26 01:04:58.289217 Evaluation finished


In [38]:
def save_ground_truths_as_local_model_predictions(raw_data, docs_dict):
    predictions = []
    for row in raw_data:
        mention_id, entity = eval(row[0])
        if row[-1]:
            predictions.append((mention_id, entity))
        
    print('Number of groups:', len(predictions))
    
    doc_id_q_ent_lists_dict = defaultdict(list)
    for q_ent in predictions:
        doc_id_q_ent_lists_dict[docs_dict[q_ent[0]]].append(q_ent)
    
    for key, vals in doc_id_q_ent_lists_dict.items():
        conn.hset('doc-predicted-ents-coref-new-for-review', key, json.dumps(vals))

In [48]:
save_ground_truths_as_local_model_predictions(d_train_ctx_raw, q_docs_dict)

Number of groups: 18288


In [51]:
d_train_coh_gt = fetch_all_features('basic_fea_coh_gt', 'train', 'aida_conll')
print(d_train_coh_gt.shape)

(531835, 24)
(136259, 24)
(131893, 24)


In [52]:
d_train_ctx_coref_coh_gt  = combine_features(d_train_ctx_coref, d_train_coh_gt)
print(d_train_ctx_coref_coh_gt.shape)

Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
(531835, 54)
(136259, 54)
(131893, 54)


In [68]:
process('ctx_coref_coh_gt', 
        d_train_ctx_coref_coh_gt, d_test_a_ctx_coref_coh, d_test_b_ctx_coref_coh, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-26 17:26:31.328742 Start training
2019-08-26 17:42:02.539844 Training finished
2019-08-26 17:42:02.540337 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9819319346313575, corr_num: 18206
n_estimators: 4900, max_depth: 6, acc_validation: 0.8956376539344605, corr_num: 4291
n_estimators: 4900, max_depth: 6, acc_test: 0.8900780379041249, corr_num: 3992
2019-08-26 17:42:09.484751 Evaluation finished
2019-08-26 17:42:09.533159 Model Saved: 4900_6_ctx_coref_coh_gt


In [69]:
model = load_model('4900_6_ctx_coref')
save_local_model_predictions(model, 
                             d_train_ctx_coref, d_train_ctx_raw, 
                             d_test_a_ctx_coref, d_test_a_ctx_raw,  
                             d_test_b_ctx_coref, d_test_b_ctx_raw, top_k=3)

test_a_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testa'))
test_b_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testb'))
print('test_a_filter', len(test_a_filter))
print('test_b_filter', len(test_b_filter))

process('ctx_coref_coh_gt', d_train_ctx_coref_coh_gt, d_test_a_ctx_coref_coh, d_test_b_ctx_coref_coh, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6], train_mode=False, 
        test_a_filter=test_a_filter, test_b_filter=test_b_filter)

Number of groups: 18288 4699 4413
test_a_filter 12008
test_b_filter 11383
2019-08-26 17:42:40.314359 Loading model: 4900_6_ctx_coref_coh_gt
2019-08-26 17:42:40.332039 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9819319346313575, corr_num: 18206
n_estimators: 4900, max_depth: 6, acc_validation: 0.9087873095387184, corr_num: 4354
n_estimators: 4900, max_depth: 6, acc_test: 0.8918617614269788, corr_num: 4000
2019-08-26 17:42:54.607714 Evaluation finished


In [70]:
model = load_model('4900_6_ctx_coref')
save_local_model_predictions(model, 
                             d_train_ctx_coref, d_train_ctx_raw, 
                             d_test_a_ctx_coref, d_test_a_ctx_raw,  
                             d_test_b_ctx_coref, d_test_b_ctx_raw, top_k=5)

test_a_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testa'))
test_b_filter=json.loads(conn.hget('doc-predicted-ents-top-k', 'testb'))
print('test_a_filter', len(test_a_filter))
print('test_b_filter', len(test_b_filter))

process('ctx_coref_coh_gt', d_train_ctx_coref_coh_gt, d_test_a_ctx_coref_coh, d_test_b_ctx_coref_coh, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6], train_mode=False, 
        test_a_filter=test_a_filter, test_b_filter=test_b_filter)

Number of groups: 18288 4699 4413
test_a_filter 18773
test_b_filter 17912
2019-08-26 17:43:25.270551 Loading model: 4900_6_ctx_coref_coh_gt
2019-08-26 17:43:25.287837 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9819319346313575, corr_num: 18206
n_estimators: 4900, max_depth: 6, acc_validation: 0.9119181799206846, corr_num: 4369
n_estimators: 4900, max_depth: 6, acc_test: 0.9063545150501672, corr_num: 4065
2019-08-26 17:43:39.535228 Evaluation finished


In [71]:
d_train_coh_ext = fetch_all_features('basic_fea_coh_ext', 'train', 'aida_conll')
d_test_a_coh_ext = fetch_all_features('basic_fea_coh_ext', 'testa', 'aida_conll')
d_test_b_coh_ext = fetch_all_features('basic_fea_coh_ext', 'testb', 'aida_conll')
print(d_train_coh_ext.shape)
print(d_test_a_coh_ext.shape)
print(d_test_b_coh_ext.shape)

(531835, 35)
(136259, 35)
(131893, 35)


In [72]:
d_train_ctx_coref_coh_ext  = combine_features(d_train_ctx_coref, d_train_coh_ext)
d_test_a_ctx_coref_coh_ext = combine_features(d_test_a_ctx_coref, d_test_a_coh_ext)
d_test_b_ctx_coref_coh_ext = combine_features(d_test_b_ctx_coref, d_test_b_coh_ext)
print(d_train_ctx_coref_coh_ext.shape)
print(d_test_a_ctx_coref_coh_ext.shape)
print(d_test_b_ctx_coref_coh_ext.shape)

Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
(531835, 65)
(136259, 65)
(131893, 65)


In [73]:
process('ctx_coref_coh_ext', 
        d_train_ctx_coref_coh_ext, d_test_a_ctx_coref_coh_ext, d_test_b_ctx_coref_coh_ext, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-27 12:33:57.719713 Start training
2019-08-27 14:39:52.721716 Training finished
2019-08-27 14:39:52.722342 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9761609406180897, corr_num: 18099
n_estimators: 4900, max_depth: 6, acc_validation: 0.8956376539344605, corr_num: 4291
n_estimators: 4900, max_depth: 6, acc_test: 0.8967670011148272, corr_num: 4022
2019-08-27 14:40:02.717558 Evaluation finished
2019-08-27 14:40:02.751813 Model Saved: 4900_6_ctx_coref_coh_ext


In [80]:
model = load_model('4900_6_ctx_coref')

test_a_filter = get_top_k_prediction_indices(model, d_test_a_ctx_coref, d_test_a_ctx_raw, q_docs_dict, 3)
test_b_filter = get_top_k_prediction_indices(model, d_test_b_ctx_coref, d_test_b_ctx_raw, q_docs_dict, 3)

print('testa filter', len(test_a_filter))
print('testb filter', len(test_b_filter))

process('ctx_coref_coh_ext', d_train_ctx_coref_coh_ext, d_test_a_ctx_coref_coh_ext, d_test_b_ctx_coref_coh_ext, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6], train_mode=False, 
        test_a_filter=test_a_filter, test_b_filter=test_b_filter)

/home/ymiao/.venvs/nerd/lib/python3.5/site-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


testa filter 12008
testb filter 11383
2019-08-27 16:45:27.514111 Loading model: 4900_6_ctx_coref_coh_ext
2019-08-27 16:45:27.532958 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9761609406180897, corr_num: 18099
n_estimators: 4900, max_depth: 6, acc_validation: 0.9106658317678982, corr_num: 4363
n_estimators: 4900, max_depth: 6, acc_test: 0.8918617614269788, corr_num: 4000
2019-08-27 16:45:44.049162 Evaluation finished


In [81]:
model = load_model('4900_6_ctx_coref')

test_a_filter = get_top_k_prediction_indices(model, d_test_a_ctx_coref, d_test_a_ctx_raw, q_docs_dict, 5)
test_b_filter = get_top_k_prediction_indices(model, d_test_b_ctx_coref, d_test_b_ctx_raw, q_docs_dict, 5)

print('testa filter', len(test_a_filter))
print('testb filter', len(test_b_filter))

process('ctx_coref_coh_ext', d_train_ctx_coref_coh_ext, d_test_a_ctx_coref_coh_ext, d_test_b_ctx_coref_coh_ext, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6], train_mode=False, 
        test_a_filter=test_a_filter, test_b_filter=test_b_filter)

testa filter 18773
testb filter 17912
2019-08-27 16:45:52.489350 Loading model: 4900_6_ctx_coref_coh_ext
2019-08-27 16:45:52.507812 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9761609406180897, corr_num: 18099
n_estimators: 4900, max_depth: 6, acc_validation: 0.9148403256105198, corr_num: 4383
n_estimators: 4900, max_depth: 6, acc_test: 0.9092530657748049, corr_num: 4078
2019-08-27 16:46:09.149866 Evaluation finished


In [137]:
d_train_coh_ext_global = fetch_all_features('basic_fea_coh_ext_global', 'train', 'aida_conll')
d_test_a_coh_ext_global = fetch_all_features('basic_fea_coh_ext_global', 'testa', 'aida_conll')
d_test_b_coh_ext_global = fetch_all_features('basic_fea_coh_ext_global', 'testb', 'aida_conll')
print(d_train_coh_ext_global.shape)
print(d_test_a_coh_ext_global.shape)
print(d_test_b_coh_ext_global.shape)
d_train_ctx_coref_coh_ext_global  = combine_features(d_train_ctx_coref, d_train_coh_ext_global)
d_test_a_ctx_coref_coh_ext_global = combine_features(d_test_a_ctx_coref, d_test_a_coh_ext_global)
d_test_b_ctx_coref_coh_ext_global = combine_features(d_test_b_ctx_coref, d_test_b_coh_ext_global)
print(d_train_ctx_coref_coh_ext_global.shape)
print(d_test_a_ctx_coref_coh_ext_global.shape)
print(d_test_b_ctx_coref_coh_ext_global.shape)

(531835, 35)
(136259, 35)
(131893, 35)
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
(531835, 65)
(136259, 65)
(131893, 65)


In [138]:
process('ctx_coref_coh_ext_global', 
        d_train_ctx_coref_coh_ext_global, d_test_a_ctx_coref_coh_ext_global, d_test_b_ctx_coref_coh_ext_global, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-28 13:56:46.325413 Start training
2019-08-28 16:28:21.910459 Training finished
2019-08-28 16:28:21.911370 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9761609406180897, corr_num: 18099
n_estimators: 4900, max_depth: 6, acc_validation: 0.8985597996242956, corr_num: 4305
n_estimators: 4900, max_depth: 6, acc_test: 0.9034559643255295, corr_num: 4052
2019-08-28 16:28:31.511281 Evaluation finished
2019-08-28 16:28:31.559428 Model Saved: 4900_6_ctx_coref_coh_ext_global


In [133]:
model = load_model('4900_6_ctx_coref')
    
for top_k in range(1, 11):
    print('=== Top K=%d ===' % top_k)
    test_a_filter = get_top_k_prediction_indices(model, d_test_a_ctx_coref, 
                                                 d_test_a_ctx_raw, q_docs_dict, top_k)
    test_b_filter = get_top_k_prediction_indices(model, d_test_b_ctx_coref, 
                                                 d_test_b_ctx_raw, q_docs_dict, top_k)

    print('test_a_filter', len(test_a_filter))
    print('test_b_filter', len(test_b_filter))

    process('ctx_coref_coh', 
            d_train_ctx_coref_coh, d_test_a_ctx_coref_coh, d_test_b_ctx_coref_coh, 
            d_train_total, d_test_a_total, d_test_b_total,
            n_estimators=[4900], max_depths=[6], train_mode=False, 
            test_a_filter=test_a_filter, test_b_filter=test_b_filter)

=== Top K=1 ===
test_a_filter 4699
test_b_filter 4413
2019-08-28 11:47:23.452617 Loading model: 4900_6_ctx_coref_coh
2019-08-28 11:47:23.478154 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9752979882422739, corr_num: 18083
n_estimators: 4900, max_depth: 6, acc_validation: 0.8874973909413484, corr_num: 4252
n_estimators: 4900, max_depth: 6, acc_test: 0.8430323299888517, corr_num: 3781
2019-08-28 11:47:44.702012 Evaluation finished
=== Top K=2 ===
test_a_filter 8458
test_b_filter 7991
2019-08-28 11:47:52.889409 Loading model: 4900_6_ctx_coref_coh
2019-08-28 11:47:52.914741 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9752979882422739, corr_num: 18083
n_estimators: 4900, max_depth: 6, acc_validation: 0.894594030473805, corr_num: 4286
n_estimators: 4900, max_depth: 6, acc_test: 0.8644370122630992, corr_num: 3877
2019-08-28 11:48:13.272880 Evaluation finished
=== Top K=3 ===
test_a_filter 12008
test_b_filter 11383
2019-08-28 11:48:20.877886 Loadin

In [134]:
model = load_model('4900_6_ctx_coref')
    
for top_k in range(11, 21):
    print('=== Top K=%d ===' % top_k)
    test_a_filter = get_top_k_prediction_indices(model, d_test_a_ctx_coref, 
                                                 d_test_a_ctx_raw, q_docs_dict, top_k)
    test_b_filter = get_top_k_prediction_indices(model, d_test_b_ctx_coref, 
                                                 d_test_b_ctx_raw, q_docs_dict, top_k)

    print('test_a_filter', len(test_a_filter))
    print('test_b_filter', len(test_b_filter))

    process('ctx_coref_coh', 
            d_train_ctx_coref_coh, d_test_a_ctx_coref_coh, d_test_b_ctx_coref_coh, 
            d_train_total, d_test_a_total, d_test_b_total,
            n_estimators=[4900], max_depths=[6], train_mode=False, 
            test_a_filter=test_a_filter, test_b_filter=test_b_filter)

=== Top K=11 ===
test_a_filter 37486
test_b_filter 36123
2019-08-28 12:07:53.267138 Loading model: 4900_6_ctx_coref_coh
2019-08-28 12:07:53.284491 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9752979882422739, corr_num: 18083
n_estimators: 4900, max_depth: 6, acc_validation: 0.8958463786265916, corr_num: 4292
n_estimators: 4900, max_depth: 6, acc_test: 0.894314381270903, corr_num: 4011
2019-08-28 12:08:14.218352 Evaluation finished
=== Top K=12 ===
test_a_filter 40426
test_b_filter 38967
2019-08-28 12:08:21.863563 Loading model: 4900_6_ctx_coref_coh
2019-08-28 12:08:21.881343 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9752979882422739, corr_num: 18083
n_estimators: 4900, max_depth: 6, acc_validation: 0.8958463786265916, corr_num: 4292
n_estimators: 4900, max_depth: 6, acc_test: 0.8940914158305463, corr_num: 4010
2019-08-28 12:08:42.344744 Evaluation finished
=== Top K=13 ===
test_a_filter 43347
test_b_filter 41789
2019-08-28 12:08:50.051169

In [135]:
model = load_model('4900_6_ctx_coref')
    
for top_k in range(21, 51):
    print('=== Top K=%d ===' % top_k)
    test_a_filter = get_top_k_prediction_indices(model, d_test_a_ctx_coref, 
                                                 d_test_a_ctx_raw, q_docs_dict, top_k)
    test_b_filter = get_top_k_prediction_indices(model, d_test_b_ctx_coref, 
                                                 d_test_b_ctx_raw, q_docs_dict, top_k)

    print('test_a_filter', len(test_a_filter))
    print('test_b_filter', len(test_b_filter))

    process('ctx_coref_coh', 
            d_train_ctx_coref_coh, d_test_a_ctx_coref_coh, d_test_b_ctx_coref_coh, 
            d_train_total, d_test_a_total, d_test_b_total,
            n_estimators=[4900], max_depths=[6], train_mode=False, 
            test_a_filter=test_a_filter, test_b_filter=test_b_filter)

=== Top K=21 ===
test_a_filter 65664
test_b_filter 63283
2019-08-28 12:30:13.948946 Loading model: 4900_6_ctx_coref_coh
2019-08-28 12:30:13.973877 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9752979882422739, corr_num: 18083
n_estimators: 4900, max_depth: 6, acc_validation: 0.8960551033187226, corr_num: 4293
n_estimators: 4900, max_depth: 6, acc_test: 0.8947603121516166, corr_num: 4013
2019-08-28 12:30:34.668116 Evaluation finished
=== Top K=22 ===
test_a_filter 68325
test_b_filter 65857
2019-08-28 12:30:42.059160 Loading model: 4900_6_ctx_coref_coh
2019-08-28 12:30:42.085716 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9752979882422739, corr_num: 18083
n_estimators: 4900, max_depth: 6, acc_validation: 0.8960551033187226, corr_num: 4293
n_estimators: 4900, max_depth: 6, acc_test: 0.8949832775919733, corr_num: 4014
2019-08-28 12:31:02.825337 Evaluation finished
=== Top K=23 ===
test_a_filter 70961
test_b_filter 68401
2019-08-28 12:31:10.53425

test_a_filter 110857
test_b_filter 106957
2019-08-28 12:38:50.950569 Loading model: 4900_6_ctx_coref_coh
2019-08-28 12:38:50.968720 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9752979882422739, corr_num: 18083
n_estimators: 4900, max_depth: 6, acc_validation: 0.8960551033187226, corr_num: 4293
n_estimators: 4900, max_depth: 6, acc_test: 0.89520624303233, corr_num: 4015
2019-08-28 12:39:12.167914 Evaluation finished
=== Top K=40 ===
test_a_filter 113238
test_b_filter 109275
2019-08-28 12:39:19.967589 Loading model: 4900_6_ctx_coref_coh
2019-08-28 12:39:19.985259 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9752979882422739, corr_num: 18083
n_estimators: 4900, max_depth: 6, acc_validation: 0.8960551033187226, corr_num: 4293
n_estimators: 4900, max_depth: 6, acc_test: 0.89520624303233, corr_num: 4015
2019-08-28 12:39:41.204216 Evaluation finished
=== Top K=41 ===
test_a_filter 115614
test_b_filter 111584
2019-08-28 12:39:48.695122 Loading model

In [136]:
def iter_num_to_suffix(num: int):
    if num == 0:
        return ''
    if num == 1:
        return '_global'
    return '_global%d' % num

global_vars = globals()
for iter_num in range(1, 5):
    iter_suffix = iter_num_to_suffix(iter_num)
    prev_iter_suffix = iter_num_to_suffix(iter_num - 1)
    
    model = load_model('4900_6_ctx_coref_coh' + prev_iter_suffix)
    
    for top_k in range(1, 11):
        print('=== Top K=%d ===' % top_k)
        test_a_filter = get_top_k_prediction_indices(model, global_vars['d_test_a_ctx_coref_coh' + prev_iter_suffix], 
                                                     d_test_a_ctx_raw, q_docs_dict, top_k)
        test_b_filter = get_top_k_prediction_indices(model, global_vars['d_test_b_ctx_coref_coh' + prev_iter_suffix], 
                                                     d_test_b_ctx_raw, q_docs_dict, top_k)

        print('test_a_filter', len(test_a_filter))
        print('test_b_filter', len(test_b_filter))

        process('ctx_coref_coh' + iter_suffix, 
                global_vars['d_train_ctx_coref_coh' + iter_suffix], 
                global_vars['d_test_a_ctx_coref_coh' + iter_suffix], 
                global_vars['d_test_b_ctx_coref_coh' + iter_suffix], 
                d_train_total, d_test_a_total, d_test_b_total,
                n_estimators=[4900], max_depths=[6], train_mode=False, 
                test_a_filter=test_a_filter, test_b_filter=test_b_filter)

=== Top K=1 ===
test_a_filter 4699
test_b_filter 4413
2019-08-28 13:19:22.606752 Loading model: 4900_6_ctx_coref_coh_global
2019-08-28 13:19:22.646731 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9756215953832048, corr_num: 18089
n_estimators: 4900, max_depth: 6, acc_validation: 0.897307451471509, corr_num: 4299
n_estimators: 4900, max_depth: 6, acc_test: 0.8963210702341137, corr_num: 4020
2019-08-28 13:19:44.017087 Evaluation finished
=== Top K=2 ===
test_a_filter 8458
test_b_filter 7991
2019-08-28 13:19:51.911850 Loading model: 4900_6_ctx_coref_coh_global
2019-08-28 13:19:51.930436 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9756215953832048, corr_num: 18089
n_estimators: 4900, max_depth: 6, acc_validation: 0.8966812773951158, corr_num: 4296
n_estimators: 4900, max_depth: 6, acc_test: 0.8981047937569677, corr_num: 4028
2019-08-28 13:20:12.339815 Evaluation finished
=== Top K=3 ===
test_a_filter 12008
test_b_filter 11383
2019-08-28 13:20:20

n_estimators: 4900, max_depth: 6, acc_test: 0.9036789297658863, corr_num: 4053
2019-08-28 13:27:51.633587 Evaluation finished
=== Top K=9 ===
test_a_filter 31478
test_b_filter 30274
2019-08-28 13:27:59.188381 Loading model: 4900_6_ctx_coref_coh_global2
2019-08-28 13:27:59.207720 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9754597918127393, corr_num: 18086
n_estimators: 4900, max_depth: 6, acc_validation: 0.8966812773951158, corr_num: 4296
n_estimators: 4900, max_depth: 6, acc_test: 0.9036789297658863, corr_num: 4053
2019-08-28 13:28:19.958914 Evaluation finished
=== Top K=10 ===
test_a_filter 34506
test_b_filter 33245
2019-08-28 13:28:27.504432 Loading model: 4900_6_ctx_coref_coh_global2
2019-08-28 13:28:27.523718 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9754597918127393, corr_num: 18086
n_estimators: 4900, max_depth: 6, acc_validation: 0.8966812773951158, corr_num: 4296
n_estimators: 4900, max_depth: 6, acc_test: 0.9036789297658863, cor

n_estimators: 4900, max_depth: 6, acc_training: 0.9755676608597164, corr_num: 18088
n_estimators: 4900, max_depth: 6, acc_validation: 0.8964725527029848, corr_num: 4295
n_estimators: 4900, max_depth: 6, acc_test: 0.9059085841694537, corr_num: 4063
2019-08-28 13:36:28.936901 Evaluation finished
=== Top K=7 ===
test_a_filter 25259
test_b_filter 24190
2019-08-28 13:36:36.818599 Loading model: 4900_6_ctx_coref_coh_global4
2019-08-28 13:36:36.835895 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9755676608597164, corr_num: 18088
n_estimators: 4900, max_depth: 6, acc_validation: 0.8964725527029848, corr_num: 4295
n_estimators: 4900, max_depth: 6, acc_test: 0.9059085841694537, corr_num: 4063
2019-08-28 13:36:58.453778 Evaluation finished
=== Top K=8 ===
test_a_filter 28394
test_b_filter 27254
2019-08-28 13:37:06.496198 Loading model: 4900_6_ctx_coref_coh_global4
2019-08-28 13:37:06.514087 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9755676608597164, 

In [92]:
def iter_num_to_suffix(num: int):
    if num == 0:
        return ''
    if num == 1:
        return '_global'
    return '_global%d' % num

global_vars = globals()
for iter_num in range(1, 5):
    iter_suffix = iter_num_to_suffix(iter_num)
    prev_iter_suffix = iter_num_to_suffix(iter_num - 1)
    
    model = load_model('4900_6_ctx_coref_coh' + prev_iter_suffix)
    
    for top_k in range(11, 21):
        print('=== Top K=%d ===' % top_k)
        test_a_filter = get_top_k_prediction_indices(model, global_vars['d_test_a_ctx_coref_coh' + prev_iter_suffix], 
                                                     d_test_a_ctx_raw, q_docs_dict, top_k)
        test_b_filter = get_top_k_prediction_indices(model, global_vars['d_test_b_ctx_coref_coh' + prev_iter_suffix], 
                                                     d_test_b_ctx_raw, q_docs_dict, top_k)

        print('test_a_filter', len(test_a_filter))
        print('test_b_filter', len(test_b_filter))

        process('ctx_coref_coh' + iter_suffix, 
                global_vars['d_train_ctx_coref_coh' + iter_suffix], 
                global_vars['d_test_a_ctx_coref_coh' + iter_suffix], 
                global_vars['d_test_b_ctx_coref_coh' + iter_suffix], 
                d_train_total, d_test_a_total, d_test_b_total,
                n_estimators=[4900], max_depths=[6], train_mode=False, 
                test_a_filter=test_a_filter, test_b_filter=test_b_filter)

=== Top K=11 ===
test_a_filter 37486
test_b_filter 36123
2019-08-27 18:19:41.434819 Loading model: 4900_6_ctx_coref_coh_global
2019-08-27 18:19:41.475043 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9756215953832048, corr_num: 18089
n_estimators: 4900, max_depth: 6, acc_validation: 0.9386349405134627, corr_num: 4497
n_estimators: 4900, max_depth: 6, acc_test: 0.9322185061315497, corr_num: 4181
2019-08-27 18:19:58.804028 Evaluation finished
=== Top K=12 ===
test_a_filter 40426
test_b_filter 38967
2019-08-27 18:20:07.368266 Loading model: 4900_6_ctx_coref_coh_global
2019-08-27 18:20:07.386636 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9756215953832048, corr_num: 18089
n_estimators: 4900, max_depth: 6, acc_validation: 0.9384262158213317, corr_num: 4496
n_estimators: 4900, max_depth: 6, acc_test: 0.9322185061315497, corr_num: 4181
2019-08-27 18:20:25.266357 Evaluation finished
=== Top K=13 ===
test_a_filter 43347
test_b_filter 41789
2019-08-27 

n_estimators: 4900, max_depth: 6, acc_test: 0.9360089186176143, corr_num: 4198
2019-08-27 18:27:32.102764 Evaluation finished
=== Top K=19 ===
test_a_filter 60285
test_b_filter 58080
2019-08-27 18:27:40.427626 Loading model: 4900_6_ctx_coref_coh_global2
2019-08-27 18:27:40.445904 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9754597918127393, corr_num: 18086
n_estimators: 4900, max_depth: 6, acc_validation: 0.936756418284283, corr_num: 4488
n_estimators: 4900, max_depth: 6, acc_test: 0.9360089186176143, corr_num: 4198
2019-08-27 18:27:58.361966 Evaluation finished
=== Top K=20 ===
test_a_filter 62980
test_b_filter 60690
2019-08-27 18:28:06.936577 Loading model: 4900_6_ctx_coref_coh_global2
2019-08-27 18:28:06.957592 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9754597918127393, corr_num: 18086
n_estimators: 4900, max_depth: 6, acc_validation: 0.936756418284283, corr_num: 4488
n_estimators: 4900, max_depth: 6, acc_test: 0.9360089186176143, corr

n_estimators: 4900, max_depth: 6, acc_training: 0.9755676608597164, corr_num: 18088
n_estimators: 4900, max_depth: 6, acc_validation: 0.9375913170528073, corr_num: 4492
n_estimators: 4900, max_depth: 6, acc_test: 0.9386845039018952, corr_num: 4210
2019-08-27 18:35:34.644705 Evaluation finished
=== Top K=17 ===
test_a_filter 54775
test_b_filter 52787
2019-08-27 18:35:43.403473 Loading model: 4900_6_ctx_coref_coh_global4
2019-08-27 18:35:43.421477 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9755676608597164, corr_num: 18088
n_estimators: 4900, max_depth: 6, acc_validation: 0.9375913170528073, corr_num: 4492
n_estimators: 4900, max_depth: 6, acc_test: 0.9386845039018952, corr_num: 4210
2019-08-27 18:36:01.466564 Evaluation finished
=== Top K=18 ===
test_a_filter 57545
test_b_filter 55450
2019-08-27 18:36:09.877236 Loading model: 4900_6_ctx_coref_coh_global4
2019-08-27 18:36:09.895295 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9755676608597164

In [ ]:
def iter_num_to_suffix(num: int):
    if num == 0:
        return ''
    if num == 1:
        return '_global'
    return '_global%d' % num

global_vars = globals()
for iter_num in range(1, 5):
    iter_suffix = iter_num_to_suffix(iter_num)
    prev_iter_suffix = iter_num_to_suffix(iter_num - 1)
    
    model = load_model('4900_6_ctx_coref_coh' + prev_iter_suffix)
    
    for top_k in range(21, 51):
        print('=== Top K=%d ===' % top_k)
        test_a_filter = get_top_k_prediction_indices(model, global_vars['d_test_a_ctx_coref_coh' + prev_iter_suffix], 
                                                     d_test_a_ctx_raw, q_docs_dict, top_k)
        test_b_filter = get_top_k_prediction_indices(model, global_vars['d_test_b_ctx_coref_coh' + prev_iter_suffix], 
                                                     d_test_b_ctx_raw, q_docs_dict, top_k)

        print('test_a_filter', len(test_a_filter))
        print('test_b_filter', len(test_b_filter))

        process('ctx_coref_coh' + iter_suffix, 
                global_vars['d_train_ctx_coref_coh' + iter_suffix], 
                global_vars['d_test_a_ctx_coref_coh' + iter_suffix], 
                global_vars['d_test_b_ctx_coref_coh' + iter_suffix], 
                d_train_total, d_test_a_total, d_test_b_total,
                n_estimators=[4900], max_depths=[6], train_mode=False, 
                test_a_filter=test_a_filter, test_b_filter=test_b_filter)

=== Top K=21 ===
test_a_filter 65664
test_b_filter 63283
2019-08-28 00:28:41.331686 Loading model: 4900_6_ctx_coref_coh_global
2019-08-28 00:28:41.359562 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9756215953832048, corr_num: 18089
n_estimators: 4900, max_depth: 6, acc_validation: 0.9386349405134627, corr_num: 4497
n_estimators: 4900, max_depth: 6, acc_test: 0.9324414715719064, corr_num: 4182
2019-08-28 00:28:59.893939 Evaluation finished
=== Top K=22 ===
test_a_filter 68325
test_b_filter 65857
2019-08-28 00:29:09.666117 Loading model: 4900_6_ctx_coref_coh_global
2019-08-28 00:29:09.691242 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9756215953832048, corr_num: 18089
n_estimators: 4900, max_depth: 6, acc_validation: 0.9386349405134627, corr_num: 4497
n_estimators: 4900, max_depth: 6, acc_test: 0.9328874024526198, corr_num: 4184
2019-08-28 00:29:32.820072 Evaluation finished
=== Top K=23 ===
test_a_filter 70961
test_b_filter 68401
2019-08-28 

n_estimators: 4900, max_depth: 6, acc_test: 0.9328874024526198, corr_num: 4184
2019-08-28 00:38:44.593928 Evaluation finished
=== Top K=39 ===
test_a_filter 110857
test_b_filter 106957
2019-08-28 00:38:54.372904 Loading model: 4900_6_ctx_coref_coh_global
2019-08-28 00:38:54.392578 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9756215953832048, corr_num: 18089
n_estimators: 4900, max_depth: 6, acc_validation: 0.9384262158213317, corr_num: 4496
n_estimators: 4900, max_depth: 6, acc_test: 0.9331103678929766, corr_num: 4185
2019-08-28 00:39:20.789968 Evaluation finished
=== Top K=40 ===
test_a_filter 113238
test_b_filter 109275
2019-08-28 00:39:31.075212 Loading model: 4900_6_ctx_coref_coh_global
2019-08-28 00:39:31.095234 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9756215953832048, corr_num: 18089
n_estimators: 4900, max_depth: 6, acc_validation: 0.9384262158213317, corr_num: 4496
n_estimators: 4900, max_depth: 6, acc_test: 0.9331103678929766, 

n_estimators: 4900, max_depth: 6, acc_training: 0.9754597918127393, corr_num: 18086
n_estimators: 4900, max_depth: 6, acc_validation: 0.9371738676685452, corr_num: 4490
n_estimators: 4900, max_depth: 6, acc_test: 0.9360089186176143, corr_num: 4198
2019-08-28 00:49:56.020677 Evaluation finished
=== Top K=27 ===
test_a_filter 81279
test_b_filter 78353
2019-08-28 00:50:06.997009 Loading model: 4900_6_ctx_coref_coh_global2
2019-08-28 00:50:07.016142 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9754597918127393, corr_num: 18086
n_estimators: 4900, max_depth: 6, acc_validation: 0.9371738676685452, corr_num: 4490
n_estimators: 4900, max_depth: 6, acc_test: 0.9360089186176143, corr_num: 4198
2019-08-28 00:50:31.472904 Evaluation finished
=== Top K=28 ===
test_a_filter 83817
test_b_filter 80801
2019-08-28 00:50:42.180117 Loading model: 4900_6_ctx_coref_coh_global2
2019-08-28 00:50:42.199060 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9754597918127393

test_a_filter 122628
test_b_filter 118460
2019-08-28 01:00:19.945138 Loading model: 4900_6_ctx_coref_coh_global2
2019-08-28 01:00:19.965205 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9754597918127393, corr_num: 18086
n_estimators: 4900, max_depth: 6, acc_validation: 0.936756418284283, corr_num: 4488
n_estimators: 4900, max_depth: 6, acc_test: 0.936231884057971, corr_num: 4199
2019-08-28 01:00:46.683612 Evaluation finished
=== Top K=45 ===
test_a_filter 124923
test_b_filter 120729
2019-08-28 01:00:57.366791 Loading model: 4900_6_ctx_coref_coh_global2
2019-08-28 01:00:57.386303 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9754597918127393, corr_num: 18086
n_estimators: 4900, max_depth: 6, acc_validation: 0.936756418284283, corr_num: 4488
n_estimators: 4900, max_depth: 6, acc_test: 0.936231884057971, corr_num: 4199
2019-08-28 01:01:25.753894 Evaluation finished
=== Top K=46 ===
test_a_filter 127214
test_b_filter 122983
2019-08-28 01:01:36.5875

n_estimators: 4900, max_depth: 6, acc_test: 0.9402452619843924, corr_num: 4217
2019-08-28 01:11:13.225435 Evaluation finished
=== Top K=32 ===
test_a_filter 93843
test_b_filter 90464
2019-08-28 01:11:23.742155 Loading model: 4900_6_ctx_coref_coh_global3
2019-08-28 01:11:23.763697 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9755676608597164, corr_num: 18088
n_estimators: 4900, max_depth: 6, acc_validation: 0.9409309121269046, corr_num: 4508
n_estimators: 4900, max_depth: 6, acc_test: 0.9402452619843924, corr_num: 4217
2019-08-28 01:11:49.668804 Evaluation finished
=== Top K=33 ===
test_a_filter 96317
test_b_filter 92847
2019-08-28 01:12:00.632969 Loading model: 4900_6_ctx_coref_coh_global3
2019-08-28 01:12:00.658127 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9755676608597164, corr_num: 18088
n_estimators: 4900, max_depth: 6, acc_validation: 0.9409309121269046, corr_num: 4508
n_estimators: 4900, max_depth: 6, acc_test: 0.9402452619843924, co

n_estimators: 4900, max_depth: 6, acc_training: 0.9755676608597164, corr_num: 18088
n_estimators: 4900, max_depth: 6, acc_validation: 0.9409309121269046, corr_num: 4508
n_estimators: 4900, max_depth: 6, acc_test: 0.9404682274247491, corr_num: 4218
2019-08-28 01:22:39.663466 Evaluation finished
=== Top K=50 ===
test_a_filter 136259
test_b_filter 131893
2019-08-28 01:22:50.156348 Loading model: 4900_6_ctx_coref_coh_global3
2019-08-28 01:22:50.185999 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9755676608597164, corr_num: 18088
n_estimators: 4900, max_depth: 6, acc_validation: 0.9409309121269046, corr_num: 4508
n_estimators: 4900, max_depth: 6, acc_test: 0.9404682274247491, corr_num: 4218
2019-08-28 01:23:17.624569 Evaluation finished
=== Top K=21 ===
test_a_filter 65664
test_b_filter 63283
2019-08-28 01:23:28.211888 Loading model: 4900_6_ctx_coref_coh_global4
2019-08-28 01:23:28.235133 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.97556766085971

test_a_filter 106070
test_b_filter 102287
2019-08-28 01:32:19.201908 Loading model: 4900_6_ctx_coref_coh_global4
2019-08-28 01:32:19.222641 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9755676608597164, corr_num: 18088
n_estimators: 4900, max_depth: 6, acc_validation: 0.9378000417449385, corr_num: 4493
n_estimators: 4900, max_depth: 6, acc_test: 0.938907469342252, corr_num: 4211
2019-08-28 01:32:43.674022 Evaluation finished
=== Top K=38 ===
test_a_filter 108469
test_b_filter 104625
2019-08-28 01:32:53.919511 Loading model: 4900_6_ctx_coref_coh_global4
2019-08-28 01:32:53.938331 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9755676608597164, corr_num: 18088
n_estimators: 4900, max_depth: 6, acc_validation: 0.9378000417449385, corr_num: 4493
n_estimators: 4900, max_depth: 6, acc_test: 0.938907469342252, corr_num: 4211
2019-08-28 01:33:19.130731 Evaluation finished
=== Top K=39 ===
test_a_filter 110857
test_b_filter 106957
2019-08-28 01:33:29.41

In [100]:
def iter_num_to_suffix(num: int):
    if num == 0:
        return ''
    if num == 1:
        return '_global'
    return '_global%d' % num

global_vars = globals()
for iter_num in range(4, 5):
    iter_suffix = iter_num_to_suffix(iter_num)
    prev_iter_suffix = iter_num_to_suffix(iter_num - 1)
    
    model = load_model('4900_6_ctx_coref_coh' + prev_iter_suffix)
    
    for top_k in range(46, 51):
        print('=== Top K=%d ===' % top_k)
        test_a_filter = get_top_k_prediction_indices(model, global_vars['d_test_a_ctx_coref_coh' + prev_iter_suffix], 
                                                     d_test_a_ctx_raw, q_docs_dict, top_k)
        test_b_filter = get_top_k_prediction_indices(model, global_vars['d_test_b_ctx_coref_coh' + prev_iter_suffix], 
                                                     d_test_b_ctx_raw, q_docs_dict, top_k)

        print('test_a_filter', len(test_a_filter))
        print('test_b_filter', len(test_b_filter))

        process('ctx_coref_coh' + iter_suffix, 
                global_vars['d_train_ctx_coref_coh' + iter_suffix], 
                global_vars['d_test_a_ctx_coref_coh' + iter_suffix], 
                global_vars['d_test_b_ctx_coref_coh' + iter_suffix], 
                d_train_total, d_test_a_total, d_test_b_total,
                n_estimators=[4900], max_depths=[6], train_mode=False, 
                test_a_filter=test_a_filter, test_b_filter=test_b_filter)

=== Top K=46 ===
test_a_filter 127214
test_b_filter 122983
2019-08-28 02:21:02.223642 Loading model: 4900_6_ctx_coref_coh_global4
2019-08-28 02:21:02.252691 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9755676608597164, corr_num: 18088
n_estimators: 4900, max_depth: 6, acc_validation: 0.9378000417449385, corr_num: 4493
n_estimators: 4900, max_depth: 6, acc_test: 0.938907469342252, corr_num: 4211
2019-08-28 02:21:28.011231 Evaluation finished
=== Top K=47 ===
test_a_filter 129498
test_b_filter 125225
2019-08-28 02:21:37.940905 Loading model: 4900_6_ctx_coref_coh_global4
2019-08-28 02:21:37.959363 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9755676608597164, corr_num: 18088
n_estimators: 4900, max_depth: 6, acc_validation: 0.9378000417449385, corr_num: 4493
n_estimators: 4900, max_depth: 6, acc_test: 0.938907469342252, corr_num: 4211
2019-08-28 02:22:03.900481 Evaluation finished
=== Top K=48 ===
test_a_filter 131767
test_b_filter 127458
2019-

In [172]:
def get_prediction_scores(model, data):
    data_xg = trans_data(data)
    return model.predict(data_xg)

In [173]:
model = load_model('4900_6_ctx_coref')
d_train_pred_score = np.vstack([d_train_ctx[:, 0], 523332
                                get_prediction_scores(mod4el, d_train_ctx_coref), 
                                d_train_ctx[:, -1]]).T
d_test_a_pred_score = np.vstack([d_test_a_ctx[:, 0], 
                                get_prediction_scores(model, d_test_a_ctx_coref), 
                                d_test_a_ctx[:, -1]]).T
d_test_b_pred_score = np.vstack([d_test_b_ctx[:, 0], 
                                get_prediction_scores(model, d_test_b_ctx_coref), 
                                d_test_b_ctx[:, -1]]).T

print(d_train_pred_score.shape)
print(d_test_a_pred_score.shape)
print(d_test_b_pred_score.shape)

(531835, 3)
(136259, 3)
(131893, 3)


In [165]:
print(d_train_ctx[0][0] == 371183)

True


In [166]:
d_train_coh_ext3 = fetch_all_features('basic_fea_coh_ext3', 'train', 'aida_conll')
d_test_a_coh_ext3 = fetch_all_features('basic_fea_coh_ext3', 'testa', 'aida_conll')
d_test_b_coh_ext3 = fetch_all_features('basic_fea_coh_ext3', 'testb', 'aida_conll')
print(d_train_coh_ext3.shape)
print(d_test_a_coh_ext3.shape)
print(d_test_b_coh_ext3.shape)
d_train_ctx_coref_coh_ext3  = combine_features(d_train_ctx_coref, d_train_coh_ext3)
d_test_a_ctx_coref_coh_ext3 = combine_features(d_test_a_ctx_coref, d_test_a_coh_ext3)
d_test_b_ctx_coref_coh_ext3 = combine_features(d_test_b_ctx_coref, d_test_b_coh_ext3)
print(d_train_ctx_coref_coh_ext3.shape)
print(d_test_a_ctx_coref_coh_ext3.shape)
print(d_test_b_ctx_coref_coh_ext3.shape)

(531835, 35)
(136259, 35)
(131893, 35)
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
(531835, 65)
(136259, 65)
(131893, 65)


In [167]:
process('ctx_coref_coh_ext3', 
        d_train_ctx_coref_coh_ext3, d_test_a_ctx_coref_coh_ext3, d_test_b_ctx_coref_coh_ext3, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

/home/ymiao/.venvs/nerd/lib/python3.5/site-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


2019-08-29 13:12:18.471128 Start training
2019-08-29 13:32:53.164550 Training finished
2019-08-29 13:32:53.165028 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9754597918127393, corr_num: 18086
n_estimators: 4900, max_depth: 6, acc_validation: 0.9037779169275725, corr_num: 4330
n_estimators: 4900, max_depth: 6, acc_test: 0.8996655518394648, corr_num: 4035
2019-08-29 13:33:01.436330 Evaluation finished
2019-08-29 13:33:01.468096 Model Saved: 4900_6_ctx_coref_coh_ext3


In [174]:
d_train_ctx_coref_coh_and_pred_score  = combine_features(d_train_ctx_coref_coh, d_train_pred_score)
d_test_a_ctx_coref_coh_and_pred_score = combine_features(d_test_a_ctx_coref_coh, d_test_a_pred_score)
d_test_b_ctx_coref_coh_and_pred_score = combine_features(d_test_b_ctx_coref_coh, d_test_b_pred_score)
print(d_train_ctx_coref_coh_and_pred_score.shape)
print(d_test_a_ctx_coref_coh_and_pred_score.shape)
print(d_test_b_ctx_coref_coh_and_pred_score.shape)

Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
(531835, 55)
(136259, 55)
(131893, 55)


In [175]:
process('ctx_coref_coh_orig_and_pred_score', 
        d_train_ctx_coref_coh_and_pred_score, 
        d_test_a_ctx_coref_coh_and_pred_score, 
        d_test_b_ctx_coref_coh_and_pred_score, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-29 17:18:44.776039 Start training
2019-08-29 19:26:26.705761 Training finished
2019-08-29 19:26:26.706374 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9761609406180897, corr_num: 18099
n_estimators: 4900, max_depth: 6, acc_validation: 0.894594030473805, corr_num: 4286
n_estimators: 4900, max_depth: 6, acc_test: 0.862876254180602, corr_num: 3870
2019-08-29 19:26:34.382995 Evaluation finished
2019-08-29 19:26:34.419586 Model Saved: 4900_6_ctx_coref_coh_orig_and_pred_score


In [25]:
d_train_ctx_can_fixed = fetch_all_features('basic_ctx_can_fixed', 'train', 'aida_conll')
d_test_a_ctx_can_fixed = fetch_all_features('basic_ctx_can_fixed', 'testa', 'aida_conll')
d_test_b_ctx_can_fixed = fetch_all_features('basic_ctx_can_fixed', 'testb', 'aida_conll')
print(d_train_ctx_can_fixed.shape)
print(d_test_a_ctx_can_fixed.shape)
print(d_test_b_ctx_can_fixed.shape)

d_train_coref_can_fixed = fetch_all_features('basic_coref_can_fixed', 'train', 'aida_conll')
d_test_a_coref_can_fixed = fetch_all_features('basic_coref_can_fixed', 'testa', 'aida_conll')
d_test_b_coref_can_fixed = fetch_all_features('basic_coref_can_fixed', 'testb', 'aida_conll')
print(d_train_coref_can_fixed.shape)
print(d_test_a_coref_can_fixed.shape)
print(d_test_b_coref_can_fixed.shape)

d_train_ctx_coref_can_fixed = combine_features(d_train_ctx_can_fixed, d_train_coref_can_fixed)
d_test_a_ctx_coref_can_fixed = combine_features(d_test_a_ctx_can_fixed, d_test_a_coref_can_fixed)
d_test_b_ctx_coref_can_fixed = combine_features(d_test_b_ctx_can_fixed, d_test_b_coref_can_fixed)
print(d_train_ctx_coref_can_fixed.shape)
print(d_test_a_ctx_coref_can_fixed.shape)
print(d_test_b_ctx_coref_can_fixed.shape)

(531841, 21)
(136260, 21)
(131893, 21)
(531841, 13)
(136260, 13)
(131893, 13)
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
(531841, 32)
(136260, 32)
(131893, 32)


In [182]:
process('ctx_coref', 
        d_train_ctx_coref_can_fixed, 
        d_test_a_ctx_coref_can_fixed, 
        d_test_b_ctx_coref_can_fixed, 
        d_train_total, d_test_a_total, d_test_b_total, train_mode=False,
        n_estimators=[4900], max_depths=[6])

2019-08-29 22:48:03.515405 Loading model: 4900_6_ctx_coref
2019-08-29 22:48:03.533444 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.960519928806429, corr_num: 17809
n_estimators: 4900, max_depth: 6, acc_validation: 0.8864537674806929, corr_num: 4247
n_estimators: 4900, max_depth: 6, acc_test: 0.841917502787068, corr_num: 3776
2019-08-29 22:48:25.779701 Evaluation finished


In [26]:
d_train_coh_can_fixed = fetch_all_features('basic_coh_can_fixed', 'train', 'aida_conll')
d_test_a_coh_can_fixed = fetch_all_features('basic_coh_can_fixed', 'testa', 'aida_conll')
d_test_b_coh_can_fixed = fetch_all_features('basic_coh_can_fixed', 'testb', 'aida_conll')
print(d_train_coh_can_fixed.shape)
print(d_test_a_coh_can_fixed.shape)
print(d_test_b_coh_can_fixed.shape)

d_train_ctx_coref_coh_can_fixed = combine_features(d_train_ctx_coref_can_fixed, d_train_coh_can_fixed)
d_test_a_ctx_coref_coh_can_fixed = combine_features(d_test_a_ctx_coref_can_fixed, d_test_a_coh_can_fixed)
d_test_b_ctx_coref_coh_can_fixed = combine_features(d_test_b_ctx_coref_can_fixed, d_test_b_coh_can_fixed)
print(d_train_ctx_coref_coh_can_fixed.shape)
print(d_test_a_ctx_coref_coh_can_fixed.shape)
print(d_test_b_ctx_coref_coh_can_fixed.shape)

(531841, 24)
(136260, 24)
(131893, 24)
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
(531841, 54)
(136260, 54)
(131893, 54)


In [27]:
process('ctx_coref_coh', 
        d_train_ctx_coref_coh_can_fixed, 
        d_test_a_ctx_coref_coh_can_fixed, 
        d_test_b_ctx_coref_coh_can_fixed, 
        d_train_total, d_test_a_total, d_test_b_total, train_mode=False,
        n_estimators=[4900], max_depths=[6])

/home/ymiao/.venvs/nerd/lib/python3.5/site-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


2019-08-30 12:45:14.544124 Loading model: 4900_6_ctx_coref_coh
2019-08-30 12:45:14.582684 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9754058572892509, corr_num: 18085
n_estimators: 4900, max_depth: 6, acc_validation: 0.8962638280108537, corr_num: 4294
n_estimators: 4900, max_depth: 6, acc_test: 0.89520624303233, corr_num: 4015
2019-08-30 12:45:34.113089 Evaluation finished


In [183]:
process('ctx_coref_can_fixed', 
        d_train_ctx_coref_can_fixed, 
        d_test_a_ctx_coref_can_fixed, 
        d_test_b_ctx_coref_can_fixed, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-29 22:59:10.442056 Start training
2019-08-30 00:56:05.647429 Training finished
2019-08-30 00:56:05.647613 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9603041907124751, corr_num: 17805
n_estimators: 4900, max_depth: 6, acc_validation: 0.8864537674806929, corr_num: 4247
n_estimators: 4900, max_depth: 6, acc_test: 0.842809364548495, corr_num: 3780
2019-08-30 00:56:13.768130 Evaluation finished
2019-08-30 00:56:13.808888 Model Saved: 4900_6_ctx_coref_can_fixed


In [28]:
d_train_coh_can_fixed_retrained = fetch_all_features('basic_fea_coh_can_fixed_retrained', 'train', 'aida_conll')
d_test_a_coh_can_fixed_retrained = fetch_all_features('basic_fea_coh_can_fixed_retrained', 'testa', 'aida_conll')
d_test_b_coh_can_fixed_retrained = fetch_all_features('basic_fea_coh_can_fixed_retrained', 'testb', 'aida_conll')
print(d_train_coh_can_fixed_retrained.shape)
print(d_test_a_coh_can_fixed_retrained.shape)
print(d_test_b_coh_can_fixed_retrained.shape)

d_train_ctx_coref_coh_can_fixed_retrained = combine_features(d_train_ctx_coref_can_fixed, d_train_coh_can_fixed_retrained)
d_test_a_ctx_coref_coh_can_fixed_retrained = combine_features(d_test_a_ctx_coref_can_fixed, d_test_a_coh_can_fixed_retrained)
d_test_b_ctx_coref_coh_can_fixed_retrained = combine_features(d_test_b_ctx_coref_can_fixed, d_test_b_coh_can_fixed_retrained)
print(d_train_ctx_coref_coh_can_fixed_retrained.shape)
print(d_test_a_ctx_coref_coh_can_fixed_retrained.shape)
print(d_test_b_ctx_coref_coh_can_fixed_retrained.shape)

(531841, 24)
(136260, 24)
(131893, 24)
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
(531841, 54)
(136260, 54)
(131893, 54)


In [29]:
process('ctx_coref_coh_can_fixed', 
        d_train_ctx_coref_coh_can_fixed_retrained, 
        d_test_a_ctx_coref_coh_can_fixed_retrained, 
        d_test_b_ctx_coref_coh_can_fixed_retrained, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-30 15:09:29.483261 Start training


KeyboardInterrupt: 

In [31]:
model = load_model('4900_6_ctx_coref')
train_filters = get_top_k_prediction_indices(model, d_train_ctx_coref, d_train_ctx_raw, q_docs_dict, 3)

# use top-k rows predicted by local models for training global model
process('ctx_coref_coh_top3', d_train_ctx_coref_coh, d_test_a_ctx_coref_coh, d_test_b_ctx_coref_coh, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6], train_filter=train_filters)

/home/ymiao/.venvs/nerd/lib/python3.5/site-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


2019-08-31 03:16:30.484111 Start training
2019-08-31 03:17:48.503374 Training finished
2019-08-31 03:17:48.503862 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9528612264710641, corr_num: 17667
n_estimators: 4900, max_depth: 6, acc_validation: 0.8718430390315174, corr_num: 4177
n_estimators: 4900, max_depth: 6, acc_test: 0.8749163879598663, corr_num: 3924
2019-08-31 03:18:07.102404 Evaluation finished
2019-08-31 03:18:07.157746 Model Saved: 4900_6_ctx_coref_coh_top3


In [33]:
d_train_ctx_local_top5 = fetch_all_features('basic_fea_ctx_local_top5', 'train', 'aida_conll')
d_test_a_ctx_local_top5 = fetch_all_features('basic_fea_ctx_local_top5', 'testa', 'aida_conll')
d_test_b_ctx_local_top5 = fetch_all_features('basic_fea_ctx_local_top5', 'testb', 'aida_conll')
print(d_train_ctx_local_top5.shape)
print(d_test_a_ctx_local_top5.shape)
print(d_test_b_ctx_local_top5.shape)

(71664, 21)
(18172, 21)
(17177, 21)


In [34]:
d_train_coref_local_top5 = fetch_all_features('basic_fea_coref_local_top5', 'train', 'aida_conll')
d_test_a_coref_local_top5 = fetch_all_features('basic_fea_coref_local_top5', 'testa', 'aida_conll')
d_test_b_coref_local_top5 = fetch_all_features('basic_fea_coref_local_top5', 'testb', 'aida_conll')
print(d_train_coref_local_top5.shape)
print(d_test_a_coref_local_top5.shape)
print(d_test_b_coref_local_top5.shape)

(71664, 13)
(18172, 13)
(17177, 13)


In [35]:
d_train_ctx_coref_local_top5 = combine_features(d_train_ctx_local_top5, d_train_coref_local_top5)
d_test_a_ctx_coref_local_top5 = combine_features(d_test_a_ctx_local_top5, d_test_a_coref_local_top5)
d_test_b_ctx_coref_local_top5 = combine_features(d_test_b_ctx_local_top5, d_test_b_coref_local_top5)
print(d_train_ctx_coref_local_top5.shape)
print(d_test_a_ctx_coref_local_top5.shape)
print(d_test_b_ctx_coref_local_top5.shape)

Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
(71664, 32)
(18172, 32)
(17177, 32)


In [41]:
process('ctx_coref_local_top5', 
        d_train_ctx_coref_local_top5, 
        d_test_a_ctx_coref_local_top5, 
        d_test_b_ctx_coref_local_top5, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-31 23:45:21.719299 Start training
2019-09-01 00:22:18.446003 Training finished
2019-09-01 00:22:18.446502 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9607896014238714, corr_num: 17814
n_estimators: 4900, max_depth: 6, acc_validation: 0.8841577958672511, corr_num: 4236
n_estimators: 4900, max_depth: 6, acc_test: 0.8401337792642141, corr_num: 3768
2019-09-01 00:22:19.675920 Evaluation finished
2019-09-01 00:22:19.711191 Model Saved: 4900_6_ctx_coref_local_top5


In [37]:
d_train_ctx_local_top5_recomputed = fetch_all_features('basic_fea_ctx_local_top5_recomputed', 'train', 'aida_conll')
d_test_a_ctx_local_top5_recomputed = fetch_all_features('basic_fea_ctx_local_top5_recomputed', 'testa', 'aida_conll')
d_test_b_ctx_local_top5_recomputed = fetch_all_features('basic_fea_ctx_local_top5_recomputed', 'testb', 'aida_conll')
print(d_train_ctx_local_top5_recomputed.shape)
print(d_test_a_ctx_local_top5_recomputed.shape)
print(d_test_b_ctx_local_top5_recomputed.shape)

(71664, 21)
(18172, 21)
(17177, 21)


In [38]:
d_train_ctx_coref_local_top5_recomputed = combine_features(d_train_ctx_local_top5_recomputed, d_train_coref_local_top5)
d_test_a_ctx_coref_local_top5_recomputed = combine_features(d_test_a_ctx_local_top5_recomputed, d_test_a_coref_local_top5)
d_test_b_ctx_coref_local_top5_recomputed = combine_features(d_test_b_ctx_local_top5_recomputed, d_test_b_coref_local_top5)
print(d_train_ctx_coref_local_top5_recomputed.shape)
print(d_test_a_ctx_coref_local_top5_recomputed.shape)
print(d_test_b_ctx_coref_local_top5_recomputed.shape)

Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
(71664, 32)
(18172, 32)
(17177, 32)


In [42]:
process('ctx_coref_local_top5_recomputed', 
        d_train_ctx_coref_local_top5_recomputed, 
        d_test_a_ctx_coref_local_top5_recomputed, 
        d_test_b_ctx_coref_local_top5_recomputed, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-09-01 00:22:19.747046 Start training
2019-09-01 00:26:37.042169 Training finished
2019-09-01 00:26:37.042669 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.969419125182029, corr_num: 17974
n_estimators: 4900, max_depth: 6, acc_validation: 0.8897933625547902, corr_num: 4263
n_estimators: 4900, max_depth: 6, acc_test: 0.854180602006689, corr_num: 3831
2019-09-01 00:26:38.234061 Evaluation finished
2019-09-01 00:26:38.272593 Model Saved: 4900_6_ctx_coref_local_top5_recomputed


In [43]:
d_train_coh_local_top5_recomputed = fetch_all_features('basic_fea_coh_local_top5_recomputed', 'train', 'aida_conll')
d_test_a_coh_local_top5_recomputed = fetch_all_features('basic_fea_coh_local_top5_recomputed', 'testa', 'aida_conll')
d_test_b_coh_local_top5_recomputed = fetch_all_features('basic_fea_coh_local_top5_recomputed', 'testb', 'aida_conll')
print(d_train_coh_local_top5_recomputed.shape)
print(d_test_a_coh_local_top5_recomputed.shape)
print(d_test_b_coh_local_top5_recomputed.shape)

(71664, 24)
(18172, 24)
(17177, 24)


In [44]:
d_train_ctx_coref_coh_local_top5_recomputed = combine_features(d_train_ctx_coref_local_top5_recomputed, d_train_coh_local_top5_recomputed)
d_test_a_ctx_coref_coh_local_top5_recomputed = combine_features(d_test_a_ctx_coref_local_top5_recomputed, d_test_a_coh_local_top5_recomputed)
d_test_b_ctx_coref_coh_local_top5_recomputed = combine_features(d_test_b_ctx_coref_local_top5_recomputed, d_test_b_coh_local_top5_recomputed)
print(d_train_ctx_coref_coh_local_top5_recomputed.shape)
print(d_test_a_ctx_coref_coh_local_top5_recomputed.shape)
print(d_test_b_ctx_coref_coh_local_top5_recomputed.shape)

Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
(71664, 54)
(18172, 54)
(17177, 54)


In [45]:
process('ctx_coref_coh_local_top5_recomputed', 
        d_train_ctx_coref_coh_local_top5_recomputed, 
        d_test_a_ctx_coref_coh_local_top5_recomputed, 
        d_test_b_ctx_coref_coh_local_top5_recomputed, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-09-01 16:36:22.278439 Start training
2019-09-01 16:38:26.329463 Training finished
2019-09-01 16:38:26.329965 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9715765061215684, corr_num: 18014
n_estimators: 4900, max_depth: 6, acc_validation: 0.8958463786265916, corr_num: 4292
n_estimators: 4900, max_depth: 6, acc_test: 0.8965440356744705, corr_num: 4021
2019-09-01 16:38:27.467853 Evaluation finished
2019-09-01 16:38:27.513430 Model Saved: 4900_6_ctx_coref_coh_local_top5_recomputed


In [48]:
print(true_labels(d_train_ctx_local_top5))
print(true_labels(d_test_a_ctx_local_top5))
print(true_labels(d_test_b_ctx_local_top5))

18158
4630
4312


In [51]:
process('ctx_coref_coh_local_top5_recomputed', 
        d_train_ctx_coref_coh_local_top5_recomputed, 
        d_test_a_ctx_coref_coh_local_top5_recomputed, 
        d_test_b_ctx_coref_coh_local_top5_recomputed, 
        d_train_total, d_test_a_total, d_test_b_total, train_mode=False,
        n_estimators=[4900], max_depths=[6], eval_func=evalerror_detail_log_xl)

/home/ymiao/.venvs/nerd/lib/python3.5/site-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


2019-09-01 16:51:10.104591 Loading model: 4900_6_ctx_coref_coh_local_top5_recomputed
2019-09-01 16:51:10.140302 Start evaluation
18154 18158 0.9791273394099563 0.9793430775039103
7.084683110633787 0.4595391193611754
n_estimators: 4900, max_depth: 6, acc_training: 0.9715765061215684, corr_num: 18014
4577 4614 0.9553329158839491 0.963055729492799
6.805830977846009 2.3499018877039295
n_estimators: 4900, max_depth: 6, acc_validation: 0.8958463786265916, corr_num: 4292
4263 4297 0.9505016722408027 0.958082497212932
6.552477043442432 2.1328936287832425
n_estimators: 4900, max_depth: 6, acc_test: 0.8965440356744705, corr_num: 4021
2019-09-01 16:51:13.608142 Evaluation finished


In [53]:
process('ctx_coref_coh', 
        d_train_ctx_coref_coh, 
        d_test_a_ctx_coref_coh, 
        d_test_b_ctx_coref_coh, 
        d_train_total, d_test_a_total, d_test_b_total, train_mode=False,
        n_estimators=[4900], max_depths=[6], eval_func=evalerror_detail_log_xl)

2019-09-01 17:03:13.094113 Loading model: 4900_6_ctx_coref_coh
2019-09-01 17:03:13.121013 Start evaluation
18283 18288 0.9860848929399709 0.9863545655574133
6.436028399506393 0.5458581280377176
n_estimators: 4900, max_depth: 6, acc_training: 0.9752979882422739, corr_num: 18083
4593 4641 0.9586725109580463 0.9686912961803381
6.195311459096339 2.27208551093936
n_estimators: 4900, max_depth: 6, acc_validation: 0.8960551033187226, corr_num: 4293
4296 4366 0.9578595317725752 0.9734671125975474
5.945393253159973 2.0214627251673596
n_estimators: 4900, max_depth: 6, acc_test: 0.89520624303233, corr_num: 4015
2019-09-01 17:03:31.947729 Evaluation finished


In [54]:
d_train_ctx_global_top3 = fetch_all_features('basic_fea_ctx_global_top3', 'train', 'aida_conll')
d_test_a_ctx_global_top3 = fetch_all_features('basic_fea_ctx_global_top3', 'testa', 'aida_conll')
d_test_b_ctx_global_top3 = fetch_all_features('basic_fea_ctx_global_top3', 'testb', 'aida_conll')
print(d_train_ctx_global_top3.shape)
print(d_test_a_ctx_global_top3.shape)
print(d_test_b_ctx_global_top3.shape)

(46467, 21)
(11756, 21)
(11043, 21)


In [56]:
d_train_coref_global_top3 = fetch_all_features('basic_fea_coref_global_top3', 'train', 'aida_conll')
d_test_a_coref_global_top3 = fetch_all_features('basic_fea_coref_global_top3', 'testa', 'aida_conll')
d_test_b_coref_global_top3 = fetch_all_features('basic_fea_coref_global_top3', 'testb', 'aida_conll')
print(d_train_coref_global_top3.shape)
print(d_test_a_coref_global_top3.shape)
print(d_test_b_coref_global_top3.shape)

(46467, 13)
(11756, 13)
(11043, 13)


In [57]:
d_train_ctx_coref_global_top3 = combine_features(d_train_ctx_global_top3, d_train_coref_global_top3)
d_test_a_ctx_coref_global_top3 = combine_features(d_test_a_ctx_global_top3, d_test_a_coref_global_top3)
d_test_b_ctx_coref_global_top3 = combine_features(d_test_b_ctx_global_top3, d_test_b_coref_global_top3)
print(d_train_ctx_coref_global_top3.shape)
print(d_test_a_ctx_coref_global_top3.shape)
print(d_test_b_ctx_coref_global_top3.shape)

Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
(46467, 32)
(11756, 32)
(11043, 32)


In [58]:
process('ctx_coref_global_top3', 
        d_train_ctx_coref_global_top3, 
        d_test_a_ctx_coref_global_top3, 
        d_test_b_ctx_coref_global_top3, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

/home/ymiao/.venvs/nerd/lib/python3.5/site-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


2019-09-01 18:00:14.719965 Start training
2019-09-01 18:01:03.894624 Training finished
2019-09-01 18:01:03.895503 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.969526994229006, corr_num: 17976
n_estimators: 4900, max_depth: 6, acc_validation: 0.8874973909413484, corr_num: 4252
n_estimators: 4900, max_depth: 6, acc_test: 0.8472686733556298, corr_num: 3800
2019-09-01 18:01:04.647782 Evaluation finished
2019-09-01 18:01:04.677222 Model Saved: 4900_6_ctx_coref_global_top3


In [59]:
d_train_coh_global_top3 = fetch_all_features('basic_fea_coh_global_top3', 'train', 'aida_conll')
d_test_a_coh_global_top3 = fetch_all_features('basic_fea_coh_global_top3', 'testa', 'aida_conll')
d_test_b_coh_global_top3 = fetch_all_features('basic_fea_coh_global_top3', 'testb', 'aida_conll')
print(d_train_coh_global_top3.shape)
print(d_test_a_coh_global_top3.shape)
print(d_test_b_coh_global_top3.shape)

(46467, 24)
(11756, 24)
(11043, 24)


In [60]:
d_train_ctx_coref_coh_global_top3 = combine_features(d_train_ctx_coref_global_top3, d_train_coh_global_top3)
d_test_a_ctx_coref_coh_global_top3 = combine_features(d_test_a_ctx_coref_global_top3, d_test_a_coh_global_top3)
d_test_b_ctx_coref_coh_global_top3 = combine_features(d_test_b_ctx_coref_global_top3, d_test_b_coh_global_top3)
print(d_train_ctx_coref_coh_global_top3.shape)
print(d_test_a_ctx_coref_coh_global_top3.shape)
print(d_test_b_ctx_coref_coh_global_top3.shape)

Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
(46467, 54)
(11756, 54)
(11043, 54)


In [61]:
process('ctx_coref_coh_global_top3', 
        d_train_ctx_coref_coh_global_top3, 
        d_test_a_ctx_coref_coh_global_top3, 
        d_test_b_ctx_coref_coh_global_top3, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-09-01 18:19:01.208889 Start training
2019-09-01 18:20:20.765258 Training finished
2019-09-01 18:20:20.765764 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9724394584973842, corr_num: 18030
n_estimators: 4900, max_depth: 6, acc_validation: 0.8931329576288874, corr_num: 4279
n_estimators: 4900, max_depth: 6, acc_test: 0.893422519509476, corr_num: 4007
2019-09-01 18:20:21.516661 Evaluation finished
2019-09-01 18:20:21.542425 Model Saved: 4900_6_ctx_coref_coh_global_top3


In [63]:
process('ctx_coref_coh_global_top3', 
        d_train_ctx_coref_coh_global_top3, 
        d_test_a_ctx_coref_coh_global_top3, 
        d_test_b_ctx_coref_coh_global_top3, 
        d_train_total, d_test_a_total, d_test_b_total, train_mode=False,
        n_estimators=[4900], max_depths=[6], eval_func=evalerror_detail_log_xl)

/home/ymiao/.venvs/nerd/lib/python3.5/site-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


2019-09-01 18:31:54.127177 Loading model: 4900_6_ctx_coref_coh_global_top3
2019-09-01 18:31:54.159686 Start evaluation
18156 18158 0.9792352084569332 0.9793430775039103
6.927968150472532 0.4268307213205844
n_estimators: 4900, max_depth: 6, acc_training: 0.9724394584973842, corr_num: 18030
4582 4614 0.9563765393446044 0.963055729492799
6.659764653868764 2.466221240029406
n_estimators: 4900, max_depth: 6, acc_validation: 0.8931329576288874, corr_num: 4279
4259 4297 0.9496098104793756 0.958082497212932
6.4027682420096514 2.0277612530979616
n_estimators: 4900, max_depth: 6, acc_test: 0.893422519509476, corr_num: 4007
2019-09-01 18:31:57.909345 Evaluation finished


In [67]:
d_train_ctx_global_top5 = fetch_all_features('basic_fea_ctx_global_top5', 'train', 'aida_conll')
d_test_a_ctx_global_top5 = fetch_all_features('basic_fea_ctx_global_top5', 'testa', 'aida_conll')
d_test_b_ctx_global_top5 = fetch_all_features('basic_fea_ctx_global_top5', 'testb', 'aida_conll')
print(d_train_ctx_global_top5.shape)
print(d_test_a_ctx_global_top5.shape)
print(d_test_b_ctx_global_top5.shape)

d_train_coref_global_top5 = fetch_all_features('basic_fea_coref_global_top5', 'train', 'aida_conll')
d_test_a_coref_global_top5 = fetch_all_features('basic_fea_coref_global_top5', 'testa', 'aida_conll')
d_test_b_coref_global_top5 = fetch_all_features('basic_fea_coref_global_top5', 'testb', 'aida_conll')
print(d_train_coref_global_top5.shape)
print(d_test_a_coref_global_top5.shape)
print(d_test_b_coref_global_top5.shape)

d_train_ctx_coref_global_top5 = combine_features(d_train_ctx_global_top5, d_train_coref_global_top5)
d_test_a_ctx_coref_global_top5 = combine_features(d_test_a_ctx_global_top5, d_test_a_coref_global_top5)
d_test_b_ctx_coref_global_top5 = combine_features(d_test_b_ctx_global_top5, d_test_b_coref_global_top5)
print(d_train_ctx_coref_global_top5.shape)
print(d_test_a_ctx_coref_global_top5.shape)
print(d_test_b_ctx_coref_global_top5.shape)

(71664, 21)
(18172, 21)
(17177, 21)
(71664, 13)
(18172, 13)
(17177, 13)
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
(71664, 32)
(18172, 32)
(17177, 32)


In [68]:
process('ctx_coref_global_top5', 
        d_train_ctx_coref_global_top5, 
        d_test_a_ctx_coref_global_top5, 
        d_test_b_ctx_coref_global_top5, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-09-01 21:15:04.543463 Start training
2019-09-01 21:16:19.847759 Training finished
2019-09-01 21:16:19.848264 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9695809287524945, corr_num: 17977
n_estimators: 4900, max_depth: 6, acc_validation: 0.8879148403256105, corr_num: 4254
n_estimators: 4900, max_depth: 6, acc_test: 0.8539576365663322, corr_num: 3830
2019-09-01 21:16:20.928129 Evaluation finished
2019-09-01 21:16:20.965270 Model Saved: 4900_6_ctx_coref_global_top5


In [69]:
d_train_coh_global_top5 = fetch_all_features('basic_fea_coh_global_top5', 'train', 'aida_conll')
d_test_a_coh_global_top5 = fetch_all_features('basic_fea_coh_global_top5', 'testa', 'aida_conll')
d_test_b_coh_global_top5 = fetch_all_features('basic_fea_coh_global_top5', 'testb', 'aida_conll')
print(d_train_coh_global_top5.shape)
print(d_test_a_coh_global_top5.shape)
print(d_test_b_coh_global_top5.shape)

d_train_ctx_coref_coh_global_top5 = combine_features(d_train_ctx_coref_global_top5, d_train_coh_global_top5)
d_test_a_ctx_coref_coh_global_top5 = combine_features(d_test_a_ctx_coref_global_top5, d_test_a_coh_global_top5)
d_test_b_ctx_coref_coh_global_top5 = combine_features(d_test_b_ctx_coref_global_top5, d_test_b_coh_global_top5)
print(d_train_ctx_coref_coh_global_top5.shape)
print(d_test_a_ctx_coref_coh_global_top5.shape)
print(d_test_b_ctx_coref_coh_global_top5.shape)

(71664, 24)
(18172, 24)
(17177, 24)
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
(71664, 54)
(18172, 54)
(17177, 54)


In [70]:
process('ctx_coref_coh_global_top5', 
        d_train_ctx_coref_coh_global_top5, 
        d_test_a_ctx_coref_coh_global_top5, 
        d_test_b_ctx_coref_coh_global_top5, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-09-01 22:18:47.823605 Start training
2019-09-01 22:20:53.947961 Training finished
2019-09-01 22:20:53.948465 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9716304406450569, corr_num: 18015
n_estimators: 4900, max_depth: 6, acc_validation: 0.897098726779378, corr_num: 4298
n_estimators: 4900, max_depth: 6, acc_test: 0.893422519509476, corr_num: 4007
2019-09-01 22:20:55.102850 Evaluation finished
2019-09-01 22:20:55.134239 Model Saved: 4900_6_ctx_coref_coh_global_top5


In [47]:
def get_wiki_articles(ids):
    prefix = 'en.wikipedia.org/wiki/'
    with conn_psql_kongzi2.cursor() as cursor:
        sql = 'select _id, text from wikipedia_articles_2014 where _id in (%s)' % (','.join(['%s'] * len(ids)))
        cursor.execute(sql, tuple(prefix + _id for _id in ids))
        return {row[0][len(prefix):]:row[1] for row in cursor.fetchall()}

In [59]:
def extract_wiki_articles(ids_file_path: str, output_dir: str, batch_size:int=500):
    def save_article(_id, text):
        text_path = os.path.join(output_dir, _id)
        if '/' in _id:  # slash exists in the id
            _dir = os.path.dirname(text_path)
            if not os.path.exists(_dir):
                os.makedirs(_dir)
        with open(text_path, 'w') as fo:
            fo.write(text)
            
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    ids = []
    with open(ids_file_path) as f:
        for line in f:
            _id = line.strip()
            ids.append(_id)
            if len(ids) == batch_size:
                for _id, text in get_wiki_articles(ids).items():
                    save_article(_id, text)
                ids = []
        if ids:
            for _id, text in get_wiki_articles(ids):
                save_article(_id, text)

In [61]:
def extract_wiki_articles_json(ids_file_path: str, output_file: str, batch_size:int=500):
    results = {}
    ids = []
    with open(ids_file_path) as f:
        for line in f:
            _id = line.strip()
            ids.append(_id)
            if len(ids) == batch_size:
                results.update(get_wiki_articles(ids))
                ids = []
        if ids:
            results.update(get_wiki_articles(ids))
    
    with open(output_file, 'w') as f:
        json.dump(results, f, indent=2)

In [62]:
extract_wiki_articles_json('/home/ymiao/candidate_list.txt', '/home/ymiao/candidate_wiki_articles.json')

In [54]:
os.path.normpath('abc\/def')

'abc\\/def'